In [2]:
%pip install pypdf2 python-dotenv langchain google-cloud-aiplatform rank_bm25 nltk faiss-cpu numpy sentence_transformers torch language_tool_python datasets bitsandbytes transformers peft pymupdf pytesseract pillow PyMuPDF transformers torch tiktoken python-multipart openai 


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip uninstall -y fitz
%pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# -*- coding: utf-8 -*-
"""mainipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1DNTN2NtuOrcD-eiQAl31Y937Wpt5vqBJ
"""

# main.py
from typing import Any
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import fitz  # PyMuPDF
import tempfile
import os
import base64
import traceback
import re
from collections import defaultdict
import math
from nltk.tokenize import sent_tokenize





# Utilidades y limpieza de texto

In [5]:
def clean_pdf_text_robust(text):
    """Limpia texto de PDF de forma MÁS robusta para RAG, atacando patrones específicos."""
    if not text: return ""
    # --- PASOS DE LIMPIEZA GENERAL ---
    ligatures = {'ﬁ': 'fi', 'ﬂ': 'fl', 'ﬀ': 'ff', 'ﬃ': 'ffi', 'ﬄ': 'ffl'}
    for lig, repl in ligatures.items(): text = text.replace(lig, repl)
    text = re.sub(r'(\w)-\s*\n\s*(\w)', r'\1\2', text) # Unir palabras con guión
    text = re.sub(r'(\w)-\s*\n\s*(\w)', r'\1\2', text) # Segunda pasada
    text = re.sub(r'^\s*Página\s+\d+(\s+de\s+\d+)?\s*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Paginación
    text = re.sub(r'\b\d+\s*/\s*\d+\b', '', text) # Paginación X / Y
    text = re.sub(r'https?://[^\s/$.?#].[^\s]*', '', text, flags=re.IGNORECASE) # URLs http/https
    text = re.sub(r'\bwww\.[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b(?!\.)', '', text) # URLs www
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text) # Emails

    # --- REGLAS ESPECÍFICAS MEJORADAS ---
    text = re.sub(r'https?://opo\.cl/[a-zA-Z0-9]+', '', text, flags=re.IGNORECASE) # URLs opo.cl
    text = re.sub(r'\bopositatest\.com\b', '', text, flags=re.IGNORECASE) # Dominio específico
    text = re.sub(r'\bv\d+\.\d+\.\d+\b', '', text, flags=re.IGNORECASE) # Versión vX.Y.Z
    text = re.sub(r'/?\s*\+34\s*(\d{1,3}\s*){2,4}', '', text) # Teléfono +34
    text = re.sub(r'^\s*\d+\s+TEMARIO\s*$', '', text, flags=re.MULTILINE | re.IGNORECASE)
    text = re.sub(r'^\s*Accede a los recursos.*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Línea recursos
    text = re.sub(r'^\s*Comprueba si tu temario.*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Línea actualizado
    text = re.sub(r'^\s*ORGANIZACIÓN DEL ESTADO\s*\|\s*TEMA\s*\d+\s*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Cabecera específica
    text = re.sub(r'^\s*RECURSOS\s*\n?\s*(GRÁFICOS)?\s*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Cabecera Recursos
    text = re.sub(r'^\s*\d+\s*$', '', text, flags=re.MULTILINE) # Líneas solo con número (experimental)
    # Eliminar bloque explicativo iconos (más agresivo)
    text = re.sub(r'^\s*RECURSOS\s+GRÁFICOS.*?simple vistazo\.', '', text, flags=re.IGNORECASE | re.DOTALL | re.MULTILINE)
    text = re.sub(r'^\s*PLAZOS\s+Sabemos que.*?simple vistazo\.', '', text, flags=re.IGNORECASE | re.DOTALL | re.MULTILINE)
    text = re.sub(r'^\s*(PLAZOS|Destacados|Pregunta de examen|Datos importantes|Negrita)\s*$', '', text, flags=re.MULTILINE | re.IGNORECASE) # Títulos sueltos iconos

    # --- PASOS DE NORMALIZACIÓN FINAL ---
    text = re.sub(r'[ \t\f\v]+', ' ', text) # Normalizar espacios horizontales
    text = re.sub(r' +\n', '\n', text) # Espacios antes de salto
    text = re.sub(r'\n +', '\n', text) # Espacios después de salto
    text = re.sub(r'\n{3,}', '\n\n', text) # Reducir saltos múltiples a 2
    text = re.sub(r'^\s*\n', '', text, flags=re.MULTILINE) # Eliminar líneas vacías residuales
    text = re.sub(r'^\s*[-•*o»·]\s+', '- ', text, flags=re.MULTILINE) # Normalizar viñetas
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', text) # Caracteres de control
    text = text.strip() # Limpiar inicio/fin
    if text: text = text.rstrip('\n') + '\n\n' # Asegurar que termine con dos saltos
    return text

# Detección de portada y secciones matemáticas

In [6]:

# --- DETECCIÓN PORTADA ---
def is_likely_cover(page_text, page_number, num_total_pages):
    """Heurística para detectar portadas."""
    lines = [line for line in page_text.split('\n') if line.strip()]
    line_count = len(lines)
    text_length = len(page_text.strip())
    if page_number == 0 and (line_count < 15 or text_length < 200):
         return True
    if page_number < 2 and line_count < 25:
        if re.search(r'\b(temario|edición|editorial|reservados todos los derechos|oposici[oó]n)\b', page_text, re.IGNORECASE):
            return True
    return False


def detectar_secciones_matematicas(texto_chunk):
    """
    Detecta menciones a secciones matemáticas típicas CUANDO APARECEN COMO
    ENCABEZADOS/INICIO DE SECCIÓN.

    Retorna un dict **SPARSE**: solo incluye claves para las secciones encontradas.
    Si no encuentra ninguna, retorna un diccionario vacío.
    """
    if not texto_chunk:
        return {} # Retorna dict vacío si no hay texto

    # --- (Copiamos la lógica de patrones y find_section_starts de la v2) ---
    def find_section_starts(pattern, text):
        matches = []
        for match in re.finditer(pattern, text):
            keyword = match.group(1) or ""
            number = match.group(2) or ""
            matches.append((keyword + number).strip())
        return sorted(list(set(matches))) if matches else None

    base_pattern = r'(?:^|\n)\s*({keyword_es}|{keyword_en})(\s+\d+(\.\d+)*)?[:\s]?\b'
    teorema_pattern = base_pattern.format(keyword_es='Teorema|Teo', keyword_en='Theorem')
    definicion_pattern = base_pattern.format(keyword_es='Definici[oó]n|Def', keyword_en='Definition')
    demostracion_pattern = base_pattern.format(keyword_es='Demostraci[oó]n|Dem|Prueba', keyword_en='Proof')
    lema_pattern = base_pattern.format(keyword_es='Lema', keyword_en='Lemma')
    proposicion_pattern = base_pattern.format(keyword_es='Proposici[oó]n|Prop', keyword_en='Proposition')
    corolario_pattern = base_pattern.format(keyword_es='Corolario|Cor', keyword_en='Corollary')
    observacion_pattern = base_pattern.format(keyword_es='Observaci[oó]n|Obs', keyword_en='Observation|Remark')
    nota_pattern = base_pattern.format(keyword_es='Nota', keyword_en='Note')
    # --- (Fin de la copia de patrones) ---

    # Detecta todas las secciones posibles
    detected_sections = {
        "teorema": find_section_starts(teorema_pattern, texto_chunk),
        "definicion": find_section_starts(definicion_pattern, texto_chunk),
        "demostracion": find_section_starts(demostracion_pattern, texto_chunk),
        "lema": find_section_starts(lema_pattern, texto_chunk),
        "proposicion": find_section_starts(proposicion_pattern, texto_chunk),
        "corolario": find_section_starts(corolario_pattern, texto_chunk),
        "observacion": find_section_starts(observacion_pattern, texto_chunk),
        "nota": find_section_starts(nota_pattern, texto_chunk),
    }

    # --- Construye el diccionario de salida dinámicamente ---
    math_sections_sparse = {}
    for section_type, mentions_list in detected_sections.items():
        if mentions_list:  # Solo si se encontró algo para este tipo
            bool_key = f"is_{section_type}"
            list_key = f"{section_type}s_mencionados" # Añade 's' para pluralizar

            # Ajuste manual para plurales irregulares si los hubiera (no aplica aquí)
            # if section_type == '...': list_key = '...'

            math_sections_sparse[bool_key] = True
            math_sections_sparse[list_key] = mentions_list

    return math_sections_sparse # Devuelve el diccionario (posiblemente vacío)


# Detección de bibliografía e imágenes

In [7]:

def detectar_paginas_resumen_biblio(pdf_path, max_paginas_finales_a_revisar=10):
    """
    Detecta páginas que contienen RESUMEN o BIBLIOGRAFÍA por separado.
    """
    paginas_resumen = []
    paginas_biblio = []

    resumen_keywords = ['RESUMEN', 'CONCLUSIÓ']
    biblio_keywords = ['BIBLIOGRAFÍA', 'REFERENCIAS', 'WEBGRAFÍA']

    try:
        doc = fitz.open(pdf_path)
        num_total_pages = len(doc)
        start_page_index = max(0, num_total_pages - max_paginas_finales_a_revisar)

        for page_num in range(start_page_index, num_total_pages):
            page = doc.load_page(page_num)
            text = page.get_text("text").upper()

            if not text or text.isspace():
                continue

            # Detectar resumen
            if any(re.search(r'(?:^[ \t]*|\n[ \t]*)' + kw + r'\b', text) for kw in resumen_keywords):
                paginas_resumen.append(page_num)

            # Detectar bibliografía
            if any(re.search(r'(?:^[ \t]*|\n[ \t]*)' + kw + r'\b', text) for kw in biblio_keywords):
                paginas_biblio.append(page_num)

        doc.close()

    except Exception as e:
        print(f"WARN (detectar_resumen_biblio): Error procesando {pdf_path}: {e}")

    return paginas_resumen, paginas_biblio


def detect_image_regions_on_page(
    page: Any,
    merge_close_distance: int = 5,
    min_area: int = 1000,
    detect_drawings: bool = False,
    debug: bool = False
) -> list:
    """
    Detecta regiones probables de imágenes (y opcionalmente dibujos vectoriales)
    en una página de PyMuPDF, retornando bounding boxes fusionadas y filtradas.

    Args:
        page (fitz.Page): Página de PyMuPDF sobre la que se detectan imágenes.
        merge_close_distance (int): Distancia máxima (en puntos) para fusionar
            bounding boxes que se solapan o están muy cerca.
        min_area (int): Área mínima (en puntos^2) para no descartar regiones pequeñas.
        detect_drawings (bool): Si True, intentará detectar regiones vectoriales
            (get_drawings()) y tratarlas como imágenes.
        debug (bool): Si True, muestra mensajes de debug.

    Returns:
        list[dict]: Lista de regiones detectadas, cada una con:
            {
              "bbox": (x0, y0, x1, y1),
              "type": "image" | "drawing"
            }
    """
    all_regions = []
    try:
        # --------------------------------------------------------
        # 1. DETECCIÓN DE IMÁGENES BITMAP
        # --------------------------------------------------------
        images_info = page.get_images(full=True)
        for img_info in images_info:
            xref = img_info[0]
            if xref == 0:
                continue  # ignorar imágenes inline o inválidas
            try:
                # Obtener los rectángulos donde se dibuja esta imagen (puede haber varios)
                img_rects = page.get_image_rects(xref)
                for rect in img_rects:
                    bbox = rect.irect  # (x0, y0, x1, y1) con coords enteras
                    x0, y0, x1, y1 = bbox
                    area = (x1 - x0) * (y1 - y0)
                    if area >= min_area:
                        all_regions.append({"bbox": bbox, "type": "image"})
                    elif debug:
                        print(f"DEBUG: Descartando imagen muy pequeña bbox={bbox}, area={area}")
            except Exception as err_rects:
                if debug:
                    print(f"DEBUG: No se pudo obtener rects de imagen xref={xref}: {err_rects}")

        # --------------------------------------------------------
        # 2. DETECCIÓN DE "DRAWINGS" VECTORIALES (OPCIONAL)
        # --------------------------------------------------------
        if detect_drawings:
            try:
                drawings = page.get_drawings()
                for d in drawings:
                    # 'type' puede ser: 'l' (line), 're' (rectangle),
                    # 'f' (fill?), 'cs' (curves?), etc.
                    # Ajusta según tus necesidades de filtrado.
                    # Aquí descartamos líneas simples:
                    if d['type'] == 'l':
                        continue
                    bbox = d['rect'].irect
                    x0, y0, x1, y1 = bbox
                    area = (x1 - x0) * (y1 - y0)
                    if area >= min_area:
                        all_regions.append({"bbox": bbox, "type": "drawing"})
                    elif debug:
                        print(f"DEBUG: Descartando dibujo pequeño bbox={bbox}, area={area}")
            except Exception as err_draw:
                if debug:
                    print(f"DEBUG: Error detectando dibujos vectoriales: {err_draw}")

        # --------------------------------------------------------
        # 3. FUSIÓN DE BBOXES CERCANOS O SOLAPADOS
        # --------------------------------------------------------
        merged_regions = _merge_bounding_boxes(all_regions, merge_close_distance, debug=debug)

        if debug:
            print(f"DEBUG: detect_image_regions_on_page => {len(all_regions)} sin fusionar, {len(merged_regions)} tras fusión")

        return merged_regions

    except Exception as e:
        print(f"WARN: Error detectando imágenes/dibujos en página: {e}")
        return []
    
def _merge_bounding_boxes(regions: list, close_dist: int, debug: bool = False) -> list:
    """
    Funde bounding boxes que se solapan o están muy cerca, retornando una
    nueva lista de regiones. Cada región es un dict con:
      { "bbox": (x0, y0, x1, y1), "type": "image" / "drawing" }.

    - close_dist: se considerará 'cerca' si la distancia entre 2 rects
      es menor o igual a close_dist.
    - Este método hace un loop iterativo hasta que no haya merges nuevos.

    Return: lista de dicts con bboxes fusionadas.
    """

    # Para fusionar rects, necesitamos una pequeña función de "check solape" y "unión"
    def rects_are_close_or_overlap(r1, r2, threshold):
        """Retorna True si r1 y r2 se solapan o la distancia entre ellos es <= threshold."""
        (x0a, y0a, x1a, y1a) = r1
        (x0b, y0b, x1b, y1b) = r2

        # 1) Si se solapan en x e y (overlap check)
        overlap_x = not (x1a < x0b or x1b < x0a)
        overlap_y = not (y1a < y0b or y1b < y0a)
        if overlap_x and overlap_y:
            return True

        # 2) Si no solapan, calculamos distancia mínima entre los rects
        #    Si es <= threshold, consideramos "cerca".
        dist = _min_dist_between_rects(r1, r2)
        return dist <= threshold

    def merge_rects(r1, r2):
        """Devuelve el bounding box que cubre ambos rects."""
        (x0a, y0a, x1a, y1a) = r1
        (x0b, y0b, x1b, y1b) = r2
        return (
            min(x0a, x0b),
            min(y0a, y0b),
            max(x1a, x1b),
            max(y1a, y1b)
        )

    changed = True
    while changed:
        changed = False
        merged_list = []
        skip_indices = set()
        n = len(regions)

        for i in range(n):
            if i in skip_indices:
                continue
            r1 = regions[i]
            merged = False
            for j in range(i+1, n):
                if j in skip_indices:
                    continue
                r2 = regions[j]

                if r1["type"] == r2["type"] or True:
                    # Si quisieras mantener separado "image" vs "drawing", podrías
                    # fusionar solo si r1["type"] == r2["type"]. O fusionar siempre.
                    if rects_are_close_or_overlap(r1["bbox"], r2["bbox"], close_dist):
                        # Merge them
                        new_bbox = merge_rects(r1["bbox"], r2["bbox"])
                        # Podríamos unificar el 'type'; aquí escogemos la del primero
                        # o creamos algo como "mixed"
                        new_type = r1["type"] if r1["type"] == r2["type"] else "mixed"
                        merged_list.append({"bbox": new_bbox, "type": new_type})
                        skip_indices.add(j)
                        merged = True
                        if debug:
                            print(f"DEBUG: Merged {r1['bbox']} + {r2['bbox']} => {new_bbox}")
                        break
            if not merged:
                # No fusionamos r1 con nadie
                merged_list.append(r1)

        if len(merged_list) < len(regions):
            # Hubo fusión => repetimos
            regions = merged_list
            changed = True
        else:
            # Sin cambio => terminamos
            regions = merged_list

    return regions


from math import sqrt

def _min_dist_between_rects(r1, r2):
    """
    Calcula la distancia mínima entre dos rects (x0, y0, x1, y1)
    si no se solapan.
    """
    x0a, y0a, x1a, y1a = r1
    x0b, y0b, x1b, y1b = r2

    # Si se solapan en x, la distancia en x es 0; de lo contrario,
    # es la diferencia entre los bordes más cercanos.
    if x1a < x0b:
        dx = x0b - x1a
    elif x1b < x0a:
        dx = x0a - x1b
    else:
        dx = 0

    # Lo mismo para y.
    if y1a < y0b:
        dy = y0b - y1a
    elif y1b < y0a:
        dy = y0a - y1b
    else:
        dy = 0

    # Distancia euclidiana
    return sqrt(dx*dx + dy*dy)


print("INFO: Funciones auxiliares STEM definidas (Fórmulas, Encabezados, Imágenes).")

INFO: Funciones auxiliares STEM definidas (Fórmulas, Encabezados, Imágenes).


# Detección de índice y fórmulas

In [8]:


def detectar_paginas_indice(pdf_path, max_paginas_a_revisar=None, umbral_min_lineas=5):
    """
    Intenta detectar las páginas del índice (Tabla de Contenido) en un PDF.

    Utiliza heurísticas mejoradas basadas en patrones de texto comunes,
    combinando numeración jerárquica, palabras clave y opcionalmente
    la presencia de números de página al final de la línea.

    Args:
        pdf_path (str): Ruta al archivo PDF.
        max_paginas_a_revisar (int): Número máximo de páginas iniciales a revisar.
        umbral_min_lineas (int): Mínimo de líneas de texto requeridas en una página
                                 para siquiera considerarla como índice.

    Returns:
        list: Una lista de índices de página (basados en 0) que probablemente
              contienen el índice. Lista vacía si no se detecta ninguno o hay error.
    """
    paginas_indice_detectadas = []
    if not os.path.exists(pdf_path):
         print(f"ERROR: (detectar_paginas_indice) Archivo no encontrado: {pdf_path}")
         return paginas_indice_detectadas

    doc = None
    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        print(f"WARN: (detectar_paginas_indice) Error al abrir PDF '{pdf_path}': {e}")
        return paginas_indice_detectadas

    # --- Heurísticas ---
    # Regex para numeración como 1., 1.1, 1.1.1., CAPÍTULO 1, TEMA 2, etc. (más flexible)
    patron_numeracion_jerarquica = re.compile(
        r"^\s*([0-9]+(\.[0-9]+)*\.?\s+|"  # 1., 1.1, 1.1.
        r"(CAP[IÍ]TULO|TEMA|SECCI[OÓ]N|PARTE)\s+[0-9IVXLCDM]+\b\.?\s*).*",
        re.IGNORECASE
    )
    # Regex para palabras clave comunes en índices/sumarios (usando search)
    patron_palabras_clave = re.compile(
        r"^\s*(INTRODUCCI[OÓ]N|PR[OÓ]LOGO|CONCLUSI[OÓ]N|EP[IÍ]LOGO|BIBLIOGRAF[IÍ]A|WEBGRAF[IÍ]A|REFERENCIAS|RESUMEN|[IÍ]NDICE|CONTENIDO|SUMARIO|ANEXO|GLOSARIO)\b",
        re.IGNORECASE
    )
    # Regex para líneas que probablemente terminan en un número de página (puede estar precedido por puntos o espacios)
    patron_linea_con_pagina = re.compile(r".*[.\s]\s*(\d+)\s*$")

    num_paginas_a_escanear = min(max_paginas_a_revisar, doc.page_count)
    posible_indice_activo = False # Flag para detectar índices multi-página

    print(f"INFO: Escaneando hasta {num_paginas_a_escanear} páginas para índice en '{os.path.basename(pdf_path)}'")

    for num_pagina in range(num_paginas_a_escanear):
        try:
            pagina = doc.load_page(num_pagina)
            # Usar bloques puede ser un poco más robusto para la separación de líneas
            bloques = pagina.get_text("blocks")
            lineas = []
            for b in bloques:
                # b[4] contiene el texto del bloque, puede tener \n internos
                block_text = b[4]
                # Dividir por nueva línea y limpiar
                lineas.extend(line.strip() for line in block_text.split('\n') if line.strip())

            num_total_lineas = len(lineas)

            # Ignorar páginas casi vacías o portadas detectadas
            if num_total_lineas < umbral_min_lineas or is_likely_cover("\n".join(lineas), num_pagina, doc.page_count):
                # print(f"DEBUG P{num_pagina+1}: Ignorada (líneas={num_total_lineas} < {umbral_min_lineas} or portada)")
                posible_indice_activo = False # Si no es índice, rompe la cadena
                continue

            contador_lineas_patron = 0
            contador_palabras_clave = 0
            contador_lineas_con_pagina = 0

            for linea in lineas:
                if patron_numeracion_jerarquica.match(linea):
                    contador_lineas_patron += 1
                # Usamos search para palabras clave, más flexible a indentación
                if patron_palabras_clave.search(linea):
                    contador_palabras_clave += 1
                if patron_linea_con_pagina.match(linea):
                    # Verificación adicional: asegurarse de que el número no sea parte de la numeración inicial
                    match_num_inicial = patron_numeracion_jerarquica.match(linea)
                    num_final_match = patron_linea_con_pagina.match(linea)
                    if num_final_match:
                         num_final_str = num_final_match.group(1)
                         # Evitar contar si el número final es el mismo que el inicial (p.ej., "1. Título 1")
                         if not (match_num_inicial and linea.strip().endswith(num_final_str) and len(linea.split()) < 4):
                              contador_lineas_con_pagina += 1


            ratio_lineas_patron = contador_lineas_patron / num_total_lineas
            ratio_lineas_con_pagina = contador_lineas_con_pagina / num_total_lineas

            # --- Lógica de Decisión Mejorada ---
            es_pagina_indice = False
            score = 0.0

            # Puntuación base por estructura de numeración (alta importancia)
            score += ratio_lineas_patron * 0.6

            # Puntuación por líneas terminando en número (media importancia)
            score += ratio_lineas_con_pagina * 0.3

            # Bonus por presencia de palabras clave (menor importancia individual, pero ayuda)
            if contador_palabras_clave > 0:
                score += 0.1 # Bonus fijo pequeño si hay al menos una
            if contador_palabras_clave > 2:
                score += 0.1 # Bonus adicional si hay varias

            # Umbral base para considerar índice
            umbral_score_base = 0.25 # Ajustar según sea necesario

            # Umbral más bajo si la página anterior fue índice (continuación)
            umbral_score_continuacion = 0.18

            if posible_indice_activo:
                if score >= umbral_score_continuacion:
                    es_pagina_indice = True
            else:
                 if score >= umbral_score_base:
                    es_pagina_indice = True

            # Refinamiento: Una página con muchas palabras clave pero CERO estructura podría ser un falso positivo
            # O una página con ALTA estructura pero pocas líneas podría no serlo.
            # (La comprobación de umbral_min_lineas ya ayuda con lo segundo)
            if contador_palabras_clave > 1 and contador_lineas_patron == 0 and contador_lineas_con_pagina == 0:
                 # Si SOLO tiene palabras clave y ninguna otra estructura, probablemente no sea índice (podría ser intro/conclusión)
                 # A menos que tenga MUCHAS líneas con palabras clave? Podría ser un índice simple.
                 if num_total_lineas > 10 and (contador_palabras_clave / num_total_lineas > 0.3): # Si >30% de lineas son keywords
                     pass # Probablemente un índice simple basado en keywords, mantener es_pagina_indice si score fue suficiente
                 else:
                     es_pagina_indice = False # Descartar si no cumple la condición anterior


            # DEBUGGING INTERNO
            print(f"  Pág {num_pagina + 1}: Lines={num_total_lineas}, "
                  f"RatioPatron={ratio_lineas_patron:.2f} ({contador_lineas_patron}), "
                  f"Keywords={contador_palabras_clave}, "
                  f"RatioPgNum={ratio_lineas_con_pagina:.2f} ({contador_lineas_con_pagina}), "
                  f"Score={score:.3f} -> Índice? {es_pagina_indice} (ActivoPrev? {posible_indice_activo})")

            if es_pagina_indice:
                if num_pagina not in paginas_indice_detectadas:
                     paginas_indice_detectadas.append(num_pagina)
                posible_indice_activo = True
            else:
                # Si la página no cumple, se rompe la posible cadena de índice
                posible_indice_activo = False

        except Exception as e:
            print(f"WARN: (detectar_paginas_indice) Error procesando página {num_pagina} del PDF: {e}")
            posible_indice_activo = False # Resetear en caso de error
            continue

    if doc:
        doc.close()

    # Post-procesamiento: a veces puede detectar una página suelta entre otras.
    # Si tenemos [0, 2], pero no 1, es menos probable que 2 sea índice.
    # Podríamos requerir bloques contiguos, pero por simplicidad lo dejamos así por ahora.

    print(f"INFO: Páginas de índice detectadas: {[p+1 for p in paginas_indice_detectadas]}") # Mostrar páginas base 1
    return paginas_indice_detectadas

def detect_formulas_in_text(text):
    """
    Detecta fórmulas tipo LaTeX en un texto (cuando existen los delimitadores):
      - Inline: $ ... $
      - Display: $$ ... $$
      - Entorno: \(...\), \[...\]
      - Entornos: \begin{equation}, \begin{align}, etc.

    Retorna lista de dicts con campos:
      {
        "latex_content": str,
        "full_match": str,
        "delimiter_type": str,  # '$', '$$', '\(\)', '\[\]', 'equation', etc.
        "start_pos": int,
        "end_pos": int
      }
    """
    # Para ignorar dólares escapados (\$), reemplazamos temporalmente
    # por algo inofensivo, de modo que no interfieran en la búsqueda.
    text_ignoring_escaped = re.sub(r'\\\$', '__ESCAPED_DOLLAR__', text)

    formulas = []

    # Patrones de LaTeX
    inline_pattern = re.compile(r'\$([^\$]+?)\$', re.DOTALL)
    display_pattern = re.compile(r'\$\$([^\$]+?)\$\$', re.DOTALL)
    backslash_paren_pattern = re.compile(r'\\\((.*?)\\\)', re.DOTALL)
    backslash_bracket_pattern = re.compile(r'\\\[(.*?)\\\]', re.DOTALL)
    begin_env_pattern = re.compile(
        r'\\begin\{(equation|align|align\*|gather|gather\*|multline|multline\*)\}(.*?)\\end\{\1\}',
        re.DOTALL
    )

    pattern_list = [
        (display_pattern, lambda m: '$$'),
        (inline_pattern, lambda m: '$'),
        (backslash_paren_pattern, lambda m: r'\(\)'),
        (backslash_bracket_pattern, lambda m: r'\[\]'),
        (begin_env_pattern, lambda m: m.group(1))
    ]

    for pattern, delim_fn in pattern_list:
        for match in pattern.finditer(text_ignoring_escaped):
            content = match.group(1).strip()
            if content:
                formulas.append({
                    "latex_content": content,
                    "full_match": match.group(0),
                    "delimiter_type": delim_fn(match),
                    "start_pos": match.start(),
                    "end_pos": match.end()
                })

    # Ordenar por posición
    formulas.sort(key=lambda f: f['start_pos'])
    return formulas

# ----------------------------------------------------------------
# 2. Detector heurístico para expresiones matemáticas “simples”
#    (cuando NO hay delimitadores LaTeX en el PDF)
# ----------------------------------------------------------------
def detect_heuristic_math_expressions(text):
    """
    Retorna True si 'text' aparenta contener “algo matemático” basado en
    patrones heurísticos MÁS REFINADOS: subíndices, superíndices, símbolos
    griegos comunes, funciones matemáticas conocidas.
    """

    # Regex para subíndices/superíndices: Mantenida igual
    subsuper_regex = re.compile(
        r'[A-Za-z0-9]\s*(\^|_)\s*(\{[^}]+\}|\w+|\d+)',
    )

    # Regex para caracteres matemáticos comunes: Mantenida igual
    # (Podrías añadir más si es necesario: ∈∉⊂⊃⊆⊇∧∨¬∀∃...)
    math_symbols_regex = re.compile(
        r'[∂∑∏√∞±≤≥≈≠÷⋅→←⇒⇔∆∫∑−∂∇αβγδεζηθικλμνξοπρστυφχψωΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩ]'
    )

    # Regex REFINADA para funciones con paréntesis:
    # - Busca funciones matemáticas comunes (sin, cos, log, exp, etc.)
    # - O busca el patrón simple f(x), g(y, z), etc. (letra seguida de paréntesis
    #   con variables/números simples dentro)
    # - Ignora mayúsculas/minúsculas para los nombres de función
    # - Es menos probable que coincida con texto descriptivo como "específicas(..)"
    common_math_funcs = r'\b(sin|cos|tan|sec|csc|cot|log|ln|exp|sqrt|abs|max|min|det|dim|ker|im|gcd|lcm|mod)\b'
    simple_func_call = r'\b[a-zA-Z]\s*\(\s*[a-zA-Z0-9_,\s]+?\s*\)' # Ej: f(x), g(a, b), h(t_1)

    function_call_regex = re.compile(f'({common_math_funcs}\s*\(.*?\)|{simple_func_call})', re.IGNORECASE)


    # Verifica si alguno de los patrones se cumple
    if subsuper_regex.search(text):
        # print("DEBUG: Found sub/superscript") # Debug opcional
        return True
    if math_symbols_regex.search(text):
        # print("DEBUG: Found math symbol") # Debug opcional
        return True
    if function_call_regex.search(text):
        # print(f"DEBUG: Found potential function call: {function_call_regex.search(text).group()}") # Debug opcional
        return True

    return False

# Extracción y limpieza principal

In [9]:

def extract_and_clean_pdf_smart(pdf_path,
                                     use_ocr_threshold=50,
                                     language='spa', # 'language' no se usa directamente aquí, quizás en OCR
                                     max_index_pages_to_scan=15,
                                     max_summary_biblio_pages_to_scan=10,
                                     debug_prints=False): # Añadido parámetro debug_prints
    """
    Extrae texto de PDF, detecta fórmulas/imágenes,
    omite portadas/índices, limpia texto, y elimina solo el bloque de
    bibliografía sin eliminar páginas enteras.

    - Aplica detect_formulas_in_text() para ver si hay LaTeX literal.
    - Marca si el texto parece contener expresiones matemáticas (heurística).
    - Detecta secciones matemáticas (teorema, definición, demostración, etc.)
      a nivel global.
    """
    all_formulas_detected = []
    all_image_regions = {}
    valid_pages_text = []
    omitted_pages_info = []
    # debug_prints = False # Se recibe como parámetro ahora

    try:
        if not os.path.exists(pdf_path):
            print(f"Error GRAVE: No se encontró el archivo PDF: {pdf_path}")
            # Devolver None o un dict vacío con error es mejor que solo None
            return {"error": f"File not found: {pdf_path}", "cleaned_text": "", "detected_formulas": [], "detected_image_regions": {}, "omitted_pages": [], "heuristic_math_detected": False, "detected_math_sections": {}}


        pdf_basename = os.path.basename(pdf_path)

        # --- Pre-detección de ÍNDICE ---
        if debug_prints: print(f"DEBUG ({pdf_basename}): Pre-detectando páginas de índice (hasta {max_index_pages_to_scan} págs)...")
        paginas_indice_detectadas = detectar_paginas_indice(
            pdf_path, max_paginas_a_revisar=max_index_pages_to_scan
        )
        if paginas_indice_detectadas:
            if debug_prints: print(f"DEBUG ({pdf_basename}): Posibles páginas de ÍNDICE (0-based): {paginas_indice_detectadas}")

        # --- Pre-detección de RESUMEN/BIBLIO ---
        if debug_prints: print(f"DEBUG ({pdf_basename}): Pre-detectando págs Resumen/Biblio (últimas {max_summary_biblio_pages_to_scan})...")
        paginas_resumen_detectadas, paginas_biblio_detectadas = detectar_paginas_resumen_biblio(
            pdf_path, max_paginas_finales_a_revisar=max_summary_biblio_pages_to_scan
        )
        if paginas_resumen_detectadas:
            if debug_prints: print(f"DEBUG ({pdf_basename}): Páginas con posible RESUMEN (0-based): {paginas_resumen_detectadas}")
        if paginas_biblio_detectadas:
            if debug_prints: print(f"DEBUG ({pdf_basename}): Páginas con posible BIBLIOGRAFÍA (0-based): {paginas_biblio_detectadas}")

        # --- Crear conjunto de páginas a omitir solo para índice ---
        # Convertir a 0-based si las funciones de detección devuelven 1-based
        # Asumiendo que devuelven 0-based:
        paginas_a_omitir_previamente = set(paginas_indice_detectadas)

        doc = fitz.open(pdf_path)
        num_total_pages = len(doc)
        if debug_prints: print(f"DEBUG ({pdf_basename}): Procesando {num_total_pages} páginas (Modo STEM).")

        for page_num in range(num_total_pages):
            page_num_real = page_num + 1 # Para logs y referencias (1-based)
            if debug_prints: print(f"DEBUG ({pdf_basename}): Procesando pág {page_num_real}/{num_total_pages}...")

            # --- Omisión de páginas índice ---
            if page_num in paginas_a_omitir_previamente:
                reason = "Índice (pre-detectado)"
                omitted_pages_info.append((page_num_real, reason))
                if debug_prints:
                    print(f"  -> OMITIDA ({reason}).")
                continue

            # --- Extracción básica de texto ---
            page_raw_text = ""
            page_raw_text_strip = ""
            try:
                page = doc.load_page(page_num) # Cargar página dentro del bucle
                page_raw_text = page.get_text("text", sort=True)
                page_raw_text_strip = page_raw_text.strip() if page_raw_text else ""
                if debug_prints and not page_raw_text_strip:
                    print(f"  -> WARN: get_text devolvió vacío o solo espacios.")
            except Exception as getTextErr:
                print(f"WARN ({pdf_basename}): get_text falló pág {page_num_real}: {getTextErr}.")
                omitted_pages_info.append((page_num_real, f"Error get_text: {getTextErr}"))
                continue # Saltar página si falla la extracción básica

            # --- Lógica de OCR (Opcional, si se requiere) ---
            used_ocr = False
            if not page_raw_text_strip or len(page_raw_text_strip) < use_ocr_threshold:
                # Aquí iría la llamada a una función OCR si decides implementarla
                # page_raw_text_ocr = apply_ocr_to_page(page, language=language)
                # if page_raw_text_ocr and len(page_raw_text_ocr.strip()) > len(page_raw_text_strip):
                #     page_raw_text = page_raw_text_ocr
                #     page_raw_text_strip = page_raw_text.strip()
                #     used_ocr = True
                #     if debug_prints: print(f"  -> INFO: OCR aplicado (resultado > {use_ocr_threshold} chars).")
                # elif debug_prints:
                #     print(f"  -> INFO: Texto < {use_ocr_threshold} chars, OCR no aplicado o sin mejora.")
                pass # Placeholder para OCR

            # --- Comprobación de texto vacío (Post-OCR si aplica) ---
            if not page_raw_text_strip:
                reason = "Sin texto válido (post-OCR)" if used_ocr else "Sin texto válido"
                omitted_pages_info.append((page_num_real, reason))
                if debug_prints: print(f"  -> OMITIDA ({reason}).")
                continue

            # --- Heurística de portada ---
            if is_likely_cover(page_raw_text_strip, page_num, num_total_pages):
                reason = "Portada (heurística)"
                omitted_pages_info.append((page_num_real, reason))
                if debug_prints: print(f"  -> OMITIDA ({reason}).")
                continue

            # --- Cortar bibliografía si corresponde (Solo si la página fue pre-detectada) ---
            final_page_text = page_raw_text_strip # Usar texto strip para búsqueda
            original_length_before_bib_cut = len(final_page_text)
            bibliography_cut_applied = False

            if page_num in paginas_biblio_detectadas:
                # Buscar keywords en mayúsculas para robustez, pero cortar el original
                biblio_keywords_regex = r'^\s*(BIBLIOGRAFÍA|REFERENCIAS|WEBGRAFÍA)\s*$' # Más específico, inicio de línea
                # Intentar buscar desde el final de la página hacia atrás podría ser más robusto
                lines = final_page_text.splitlines()
                cut_index = -1
                for i in range(len(lines) - 1, -1, -1):
                     if re.search(biblio_keywords_regex, lines[i].strip().upper()):
                         # Encontrar la posición de inicio de esta línea en el texto original
                         try:
                            cut_index = final_page_text.rindex(lines[i])
                            break
                         except ValueError:
                            pass # Seguir buscando si la línea no se encuentra exactamente

                if cut_index != -1:
                    final_page_text = final_page_text[:cut_index].strip()
                    bibliography_cut_applied = True
                    if debug_prints:
                        print(f"  -> INFO: Texto cortado por keyword de bibliografía encontrada.")

                    # Si tras cortar no queda texto, omitir la página
                    if not final_page_text:
                        reason = "Texto eliminado por contenido de bibliografía"
                        omitted_pages_info.append((page_num_real, reason))
                        if debug_prints: print(f"  -> OMITIDA ({reason}).")
                        continue

            # --- Detección de fórmulas e imágenes (en el texto final de la página) ---
            page_formulas = detect_formulas_in_text(final_page_text) # Usar texto potencialmente cortado
            if page_formulas:
                # Ajustar posiciones si se cortó la bibliografía? No, las pos son relativas a final_page_text
                all_formulas_detected.extend(page_formulas)
                if debug_prints: print(f"  -> INFO: Detectadas {len(page_formulas)} fórmulas LaTeX.")

            # Detectar imágenes (asumiendo que la función existe)
            page_image_bboxes = detect_image_regions_on_page(page)
            if page_image_bboxes:
                all_image_regions[page_num_real] = page_image_bboxes
                if debug_prints: print(f"  -> INFO: Detectadas {len(page_image_bboxes)} regiones de imagen.")

            # --- Página Aceptada (añadir texto final) ---
            valid_pages_text.append(final_page_text)
            if debug_prints: print(f"  -> ACEPTADA (len: {len(final_page_text)} chars).")

        # --- Fin del bucle de páginas ---
        doc.close()

        # --- Resumen de omisiones ---
        print("\n" + "-"*20 + f" Resumen Omisiones ({pdf_basename}) " + "-"*20)
        if not omitted_pages_info:
            print("INFO: No se omitió ninguna página.")
        else:
            omitted_by_reason = defaultdict(list)
            for page, reason in omitted_pages_info:
                omitted_by_reason[reason].append(page)
            print(f"INFO: Omitidas {len(omitted_pages_info)}/{num_total_pages} páginas:")
            for reason, pages in sorted(omitted_by_reason.items()):
                pages.sort()
                # Agrupar páginas consecutivas para mejor lectura
                grouped_pages = []
                if pages:
                    start_range = pages[0]
                    end_range = pages[0]
                    for i in range(1, len(pages)):
                        if pages[i] == end_range + 1:
                            end_range = pages[i]
                        else:
                            if start_range == end_range:
                                grouped_pages.append(str(start_range))
                            else:
                                grouped_pages.append(f"{start_range}-{end_range}")
                            start_range = end_range = pages[i]
                    # Añadir el último rango/página
                    if start_range == end_range:
                        grouped_pages.append(str(start_range))
                    else:
                        grouped_pages.append(f"{start_range}-{end_range}")
                print(f"  - Razón: '{reason}', Páginas: {', '.join(grouped_pages)}")
        print("-"*(42 + len(f" Resumen Omisiones ({pdf_basename}) ")))

        # --- Limpieza final del texto concatenado ---
        if not valid_pages_text:
            print(f"ERROR ({pdf_basename}): No se aceptó ninguna página válida.")
            return {
                "cleaned_text": "",
                "detected_formulas": [],
                "detected_image_regions": {},
                "omitted_pages": omitted_pages_info,
                "heuristic_math_detected": False,
                "detected_math_sections": {}
            }

        full_raw_text = "\n\n".join(valid_pages_text) # Unir páginas aceptadas
        if debug_prints: print(f"DEBUG ({pdf_basename}): {len(valid_pages_text)} págs aceptadas. Limpiando texto concatenado...")

        # Asumiendo que clean_pdf_text_robust está definido.
        cleaned_text = clean_pdf_text_robust(full_raw_text)
        if debug_prints: print(f"DEBUG ({pdf_basename}): Limpieza completada. Longitud final: {len(cleaned_text)} chars.")

        if not cleaned_text or cleaned_text.isspace():
            print(f"WARN ({pdf_basename}): Texto final limpio vacío.")
            # Devolver el estado aunque el texto esté vacío
            return {
                "cleaned_text": "",
                "detected_formulas": all_formulas_detected,
                "detected_image_regions": all_image_regions,
                "omitted_pages": omitted_pages_info,
                "heuristic_math_detected": False, # No hay texto para analizar
                "detected_math_sections": {}
            }

        # --- Análisis Global del Texto Limpio ---
        # Heurística global de fórmulas en el texto
        has_any_heuristic_math = detect_heuristic_math_expressions(cleaned_text)
        if debug_prints: print(f"DEBUG ({pdf_basename}): Heurística matemática global detectada: {has_any_heuristic_math}")

        # Detección de secciones matemáticas (teorema, definición, etc.)
        math_sections_info = detectar_secciones_matematicas(cleaned_text)
        if debug_prints: print(f"DEBUG ({pdf_basename}): Secciones matemáticas detectadas: {math_sections_info}")


        # --- Devolver Resultados ---
        return {
            "cleaned_text": cleaned_text,
            "detected_formulas": all_formulas_detected,       # LaTeX literal
            "detected_image_regions": all_image_regions,
            "omitted_pages": omitted_pages_info,
            "heuristic_math_detected": has_any_heuristic_math,  # True/False si hay expresiones "sospechosas"
            "detected_math_sections": math_sections_info        # Dict con las listas y booleanos
        }

    except FileNotFoundError:
 # Ser más específico con la excepción
        print(f"Error GRAVE: No se encontró el PDF: {pdf_path}")
        return {"error": f"File not found: {pdf_path}", "cleaned_text": "", "detected_formulas": [], "detected_image_regions": {}, "omitted_pages": [], "heuristic_math_detected": False, "detected_math_sections": {}}
    except Exception as e:
        print(f"Error GRAVE procesando PDF {os.path.basename(pdf_path)} ({type(e).__name__}): {e}")
        traceback.print_exc()
        return {"error": f"Processing error: {e}", "cleaned_text": "", "detected_formulas": [], "detected_image_regions": {}, "omitted_pages": [], "heuristic_math_detected": False, "detected_math_sections": {}}


# Chunking

In [10]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Asegúrate de que punkt esté descargado
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    print("Descargando 'punkt'...")
    nltk.download('punkt', quiet=True)

def chunk_text_by_sentences(text, max_words=200, overlap_sentences=1):
    """Divide el texto en chunks basados en frases, con límite de palabras y solapamiento."""
    if not text: return []
    overlap_sentences = max(0, overlap_sentences)
    try:
        sentences = sent_tokenize(text, language='spanish')
        if not sentences: # Si sent_tokenize devuelve vacío, usar fallback
             raise ValueError("sent_tokenize devolvió lista vacía")
    except Exception as e:
        # print(f"DEBUG: sent_tokenize falló o vacío ({e}), usando fallback por saltos de línea.")
        # Fallback: dividir por dobles saltos de línea, luego simples
        sentences = [p.strip() for p in text.split('\n\n') if p.strip()]
        if not sentences:
             sentences = [p.strip() for p in text.split('\n') if p.strip()]
        # Fallback final: dividir por puntos si todo lo demás falla
        if not sentences:
             sentences = [s.strip() + '.' for s in text.split('.') if s.strip()]

    chunks = []
    i = 0
    n_sentences = len(sentences)

    while i < n_sentences:
        current_chunk_sentences = []
        word_count = 0
        start_sentence_index = i

        j = i
        while j < n_sentences:
            current_sentence = sentences[j].strip()
            if not current_sentence:
                j += 1; continue

            sentence_words = len(current_sentence.split())

            # Añadir la primera frase siempre, o si cabe
            if not current_chunk_sentences or (word_count + sentence_words <= max_words):
                current_chunk_sentences.append(current_sentence)
                word_count += sentence_words
                j += 1
            else:
                break # La frase actual no cabe

        # Procesar el chunk acumulado
        if current_chunk_sentences:
            chunks.append(" ".join(current_chunk_sentences))
            # Calcular índice de inicio del siguiente chunk con solapamiento
            # Retroceder `overlap_sentences` desde `j` (la frase que no cupo)
            # pero sin ir más atrás que `start_sentence_index + 1`
            next_start_index = max(start_sentence_index + 1, j - overlap_sentences)
            i = min(next_start_index, n_sentences) # Asegurar que no se pase del final
        # Manejar caso de frase inicial demasiado larga
        elif j < n_sentences and j == i: # No se avanzó, la primera frase ya era > max_words
            long_sentence = sentences[j].strip()
            if long_sentence:
                 chunks.append(long_sentence)
                 # print(f"DEBUG: Frase inicial larga (idx {j}) añadida como chunk.")
            i = j + 1 # Avanzar al siguiente
        # Caso raro: no se añadieron frases y no era la inicial (ej, bucle infinito?)
        elif j < n_sentences:
             # print(f"DEBUG: Avanzando índice de {i} a {j} sin añadir chunk.")
             i = j
        else: # Fin de las frases
            break
    return chunks

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
%pip install --upgrade --force-reinstall \
  numpy==1.23.5 \
  pandas==1.5.3 \
  scikit-learn==1.1.3


  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scikit_learn-1.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached scipy-1.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
Using cached scikit_learn-1.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30.5 MB)
Using

In [12]:
from pprint import pprint
import fitz  # PyMuPDF
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

# Asegúrate de que punkt esté descargado
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    print("Descargando 'punkt'...")
    nltk.download('punkt', quiet=True)


# --- Código de extracción y limpieza ---
pdf_path = "/home/azureuser/cloudfiles/code/Users/papastor/Español/Biologia.pdf"
resultado = extract_and_clean_pdf_smart(pdf_path=pdf_path, debug_prints=False) # Desactiva prints internos si son muchos
texto_limpio = resultado["cleaned_text"]
topic = os.path.basename(pdf_path)

print(f"Longitud del texto limpio: {len(texto_limpio)}")
print("-" * 20)

# --- PRUEBA DIRECTA DE SENT_TOKENIZE ---
print("Probando sent_tokenize directamente...")
try:
    sentences_nltk = sent_tokenize(texto_limpio, language='spanish')
    print(f"Número de frases detectadas por sent_tokenize: {len(sentences_nltk)}")
    if sentences_nltk:
        print("Primeras 3 frases detectadas por NLTK:")
        for i, s in enumerate(sentences_nltk[:3]):
            print(f"  {i+1}: {s[:150]}...") # Muestra el inicio
        print("Últimas 3 frases detectadas por NLTK:")
        for i, s in enumerate(sentences_nltk[-3:]):
             print(f"  {-3+i}: {s[:150]}...")
    else:
        print("¡sent_tokenize devolvió una lista vacía!")

except Exception as e:
    print(f"¡Error al ejecutar sent_tokenize directamente!: {e}")

print("-" * 20)

# --- PRUEBA DIRECTA DE FALLBACKS (si sent_tokenize falla) ---
print("Probando fallbacks (split)...")
sentences_nn = [p.strip() for p in texto_limpio.split('\n\n') if p.strip()]
print(f"Bloques por '\\n\\n': {len(sentences_nn)}")
sentences_n = [p.strip() for p in texto_limpio.split('\n') if p.strip()]
print(f"Líneas por '\\n': {len(sentences_n)}")
sentences_dot = [s.strip() + '.' for s in texto_limpio.split('.') if s.strip()]
print(f"Fragmentos por '.': {len(sentences_dot)}")

print("-" * 20)


# --- Llamada a tu función de chunking (puedes comentarla mientras depuras lo anterior) ---
print("Llamando a chunk_text_by_sentences...")
chunks = chunk_text_by_sentences(texto_limpio, max_words=250, overlap_sentences=2)
print(f"Número de chunks generados: {len(chunks)}")

# Analiza cada chunk y agrega metadatos específicos
chunks_with_metadata = []
print("\nAnalizando chunks y generando metadatos...")
for i, chunk in enumerate(chunks):
    print(f"--- Procesando Chunk {i+1} ---")

    # --- Llama a las funciones refinadas ---
    detected_math_sections = detectar_secciones_matematicas(chunk) # Usa v3
    has_heuristic = detect_heuristic_math_expressions(chunk) # Usa v2
    has_latex = bool(detect_formulas_in_text(chunk))

    # Construye los metadatos base
    chunk_metadata = {
        "text": chunk,
        "has_latex_formula": has_latex,
        "has_heuristic_math": has_heuristic,
        # Inicialmente no añadimos math_sections
    }

    # --- Añade math_sections SOLO si no está vacío ---
    if detected_math_sections: # Si el diccionario tiene contenido...
        chunk_metadata["math_sections"] = detected_math_sections
        print(f"  Secciones matemáticas detectadas: {list(detected_math_sections.keys())}")
    else:
        print("  No se detectaron secciones matemáticas como encabezados.")

    # (Opcional) Puedes añadir un print para las otras detecciones
    if has_latex: print("  Se detectó LaTeX.")
    if has_heuristic: print("  Se detectó heurística matemática.")


    chunks_with_metadata.append(chunk_metadata)

# 4. Ejemplo: muestra los metadatos del primer chunk
print("=== METADATOS POR CHUNK ===")
if chunks_with_metadata:
    pprint(chunks_with_metadata[0])
else:
    print("No se generaron chunks.")

# Si quieres ver todos los metadatos de todos los chunks:
for i, chunk_meta in enumerate(chunks_with_metadata):
     print(f"\n--- Chunk {i+1} ---")
     pprint(chunk_meta)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


INFO: Escaneando hasta 15 páginas para índice en 'Biologia.pdf'
  Pág 1: Lines=25, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.84 (21), Score=0.252 -> Índice? True (ActivoPrev? False)
  Pág 2: Lines=35, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.03 (1), Score=0.009 -> Índice? False (ActivoPrev? True)
  Pág 3: Lines=48, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.02 (1), Score=0.006 -> Índice? False (ActivoPrev? False)
  Pág 4: Lines=25, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.04 (1), Score=0.012 -> Índice? False (ActivoPrev? False)
  Pág 5: Lines=53, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.04 (2), Score=0.011 -> Índice? False (ActivoPrev? False)
  Pág 6: Lines=46, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.02 (1), Score=0.007 -> Índice? False (ActivoPrev? False)
  Pág 7: Lines=50, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.02 (1), Score=0.006 -> Índice? False (ActivoPrev? False)
  Pág 8: Lines=60, RatioPatron=0.00 (0), Keywords=0, RatioPgNum=0.02 (1), Sc

In [16]:
from openai import AzureOpenAI
from dotenv import load_dotenv

dotenv_path = '/home/azureuser/cloudfiles/code/Users/papastor/.env' # O '/ruta/completa/a/.env'
loaded = load_dotenv(dotenv_path=dotenv_path)
ts_token = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_API_KEY")

# 1. Inicializa el cliente con tus credenciales y endpoint de Azure
client = AzureOpenAI(
    api_key        = ts_token,                  # tu clave de Azure
    azure_endpoint = "https://rag-opos.openai.azure.com/",  # tu endpoint
    api_version    = "2024-10-21"                           # la versión de la API que tengas desplegada
)

response = client.embeddings.create(
    model = "text-embedding-3-small",      # el nombre de tu deployment
    input = ["Aquí va el texto a embedir"]  # puede ser lista de strings
)

In [17]:
  # Install the faiss library
import faiss
import numpy as np

# 1) Inicializa tu índice FAISS
EMB_DIM = 1536
index = faiss.IndexFlatL2(EMB_DIM)

# 2) Función de embedding
def embed_batch(batch_texts):
    resp = client.embeddings.create(
        model="text-embedding-3-small",
        input=batch_texts
    )
    # Devuelve una lista de listas de floats
    return [d.embedding for d in resp.data]

# 3) Recorre en batches
BATCH_SIZE = 100
texts, metas = [], []
for i in range(0, len(chunks), BATCH_SIZE):
    batch_texts = chunks[i : i + BATCH_SIZE]
    batch_embs  = embed_batch(batch_texts)  # List[List[float]]

    # 4) Convierte TODO de golpe a un ndarray limpio
    xb = np.asarray(batch_embs, dtype=np.float32)   # shape = (N, 1536)
    xb = xb.copy(order='C')                         # fuerza orden C

    # 5) Verificaciones (puedes comentarlas después)
    assert isinstance(xb, np.ndarray)
    assert xb.dtype == np.float32
    assert xb.ndim == 2 and xb.shape[1] == EMB_DIM
    assert xb.flags['C_CONTIGUOUS']

    print(f"-- Batch {i//BATCH_SIZE}: xb.shape={xb.shape}, dtype={xb.dtype}, C_contig={xb.flags['C_CONTIGUOUS']}")

    # 6) ¡Éxito! Añádelo al índice
    index.add(xb)

    # 7) Guarda paralelamente textos y metadatos
    texts.extend(batch_texts)
    metas.extend(chunks_with_metadata[i : i + BATCH_SIZE])

# 8) Persiste tu índice y auxiliares
faiss.write_index(index, "educacion.index")
import pickle
with open("educacion_texts.pkl","wb") as f: pickle.dump(texts, f)
with open("educacion_metas.pkl","wb") as f:  pickle.dump(metas, f)

print("✅ ÍNDICE creado con", index.ntotal, "vectores")

-- Batch 0: xb.shape=(100, 1536), dtype=float32, C_contig=True
-- Batch 1: xb.shape=(9, 1536), dtype=float32, C_contig=True
✅ ÍNDICE creado con 109 vectores


In [ ]:
# --- Celda 1: Importaciones y Configuración Inicial ---

import os
import faiss
import pickle
import numpy as np
import traceback # Para imprimir errores detallados
from dotenv import load_dotenv
import re # Para la función de pesos dinámicos

# Nuevas importaciones necesarias
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder
import nltk # Opcional, para mejor tokenización
# Descomenta si usas NLTK:
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# Importaciones específicas de LangChain (mantenemos algunas para la estructura inicial)
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.documents import Document
# Ya no usaremos el objeto Langchain FAISS para buscar ni su docstore directamente aquí
# from langchain_community.vectorstores import FAISS
# from langchain_community.docstore.in_memory import InMemoryDocstore

print("INFO: Celda 1 - Importaciones cargadas.")

# Carga las variables de entorno
dotenv_path = '/home/azureuser/cloudfiles/code/Users/papastor/.env' # O '/ruta/completa/a/.env'
loaded = load_dotenv(dotenv_path=dotenv_path)
if loaded:
    print(f"INFO: Variables de entorno cargadas desde '{dotenv_path}'.")
else:
    print(f"ADVERTENCIA: No se pudo cargar el archivo .env desde '{dotenv_path}'. Asegúrate de que existe y las variables están definidas en el entorno.")

# Descarga recursos de NLTK si usas su tokenizador/stopwords (solo la primera vez)
# Descomenta las siguientes líneas si vas a usar nltk_tokenizer
# print("INFO: Verificando/descargando recursos NLTK (punkt, stopwords)...")
# try:
#     nltk.data.find('tokenizers/punkt')
# except nltk.downloader.DownloadError:
#     nltk.download('punkt')
# try:
#     nltk.data.find('corpora/stopwords')
# except nltk.downloader.DownloadError:
#     nltk.download('stopwords')
# print("INFO: Recursos NLTK listos.")

print("--- Fin Celda 1 ---")

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Celda 1 - Importaciones cargadas.
INFO: Variables de entorno cargadas desde '/home/azureuser/cloudfiles/code/Users/papastor/.env'.
--- Fin Celda 1 ---


In [ ]:
# --- Celda 2: Parámetros de Configuración y Variables Globales ---

# --- Parámetros de Archivos y Azure ---
INDEX_PATH = "educacion.index"
TEXTS_PATH = "educacion_texts.pkl"
METAS_PATH = "educacion_metas.pkl"
AZURE_EMBEDDING_DEPLOYMENT_NAME = "text-embedding-3-small" # Asegúrate que coincide con tu deployment
AZURE_API_VERSION = "2024-02-01" # O la versión que uses ej: "2023-05-15"

# --- Parámetros para Búsqueda Híbrida y Reranking ---
K_FAISS_INITIAL = 100  # Número de candidatos a recuperar de FAISS
K_BM25_INITIAL = 100   # Número de candidatos a recuperar de BM25
K_RERANK = 100         # Número de candidatos a pasar al reranker (<= K_FAISS + K_BM25)
K_FINAL = 3            # Número final de chunks a devolver al LLM
RERANKER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-12-v2' # Modelo CrossEncoder

# --- Variables Globales para inicialización única (se llenarán en la primera ejecución) ---
is_retriever_initialized = False
# Objetos principales:
embeddings_model = None
faiss_index = None
texts = None
metadatas = None
bm25 = None
reranker = None
# Opcional: stop words en español si usas NLTK
# spanish_stopwords = stopwords.words('spanish')

print("INFO: Celda 2 - Parámetros y Variables Globales definidas.")
print(f"  - K_FAISS_INITIAL: {K_FAISS_INITIAL}")
print(f"  - K_BM25_INITIAL: {K_BM25_INITIAL}")
print(f"  - K_RERANK: {K_RERANK}")
print(f"  - K_FINAL: {K_FINAL}")
print(f"  - RERANKER_MODEL: {RERANKER_MODEL}")
print("--- Fin Celda 2 ---")

INFO: Celda 2 - Parámetros y Variables Globales definidas.
  - K_FAISS_INITIAL: 100
  - K_BM25_INITIAL: 100
  - K_RERANK: 100
  - K_FINAL: 3
  - RERANKER_MODEL: cross-encoder/ms-marco-MiniLM-L-12-v2
--- Fin Celda 2 ---


In [ ]:
# --- Celda 3: Funciones Auxiliares ---

def simple_tokenizer(text):
    """Tokenizador simple: minúsculas y split por espacios."""
    if not isinstance(text, str):
        return []
    return text.lower().split()

# Opcional: Tokenizador más robusto con NLTK (requiere descargas en Celda 1)
# def nltk_tokenizer(text):
#     """Tokenizador con NLTK: minúsculas, palabras, sin puntuación ni stopwords."""
#     if not isinstance(text, str):
#         return []
#     words = word_tokenize(text.lower(), language='spanish')
#     # Asegúrate que spanish_stopwords está definida si descomentas esto
#     # return [word for word in words if word.isalnum() and word not in spanish_stopwords]
#     return [word for word in words if word.isalnum()] # Sin stopwords

# Elige tu tokenizador preferido aquí (¡asegúrate que la función existe!)
tokenizer_for_bm25 = simple_tokenizer
# tokenizer_for_bm25 = nltk_tokenizer # Si prefieres NLTK


def calcular_pesos_dinamicos(query: str) -> tuple[float, float]:
    """
    Analiza la query educativa y ajusta pesos entre BM25 y Embeddings.
    Devuelve (peso_bm25, peso_emb).
    Prioriza BM25 para queries específicas (nombres, fechas, términos técnicos, definiciones literales).
    Prioriza Embeddings para queries conceptuales (cómo, por qué, explica, relaciona, compara).
    """
    query_lower = query.lower()
    query_original = query # Para checks de mayúsculas si es necesario

    # --- Indicadores de Especificidad (Favorecen BM25) ---
    peso_bm25 = 0.4 # Peso base (default)
    peso_emb = 0.6  # Peso base (default)
    razon = ["Default -> Ligero sesgo Embedding"]

    # 1. Presencia de Nombres Propios (heurística simple: palabras con mayúscula inicial, no al inicio de frase)
    #    O secuencias de palabras con mayúsculas. Más útil para nombres propios compuestos.
    #    (Podría requerir un NER más sofisticado para precisión, pero esto es una aproximación)
    if re.search(r'\b[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+\s+[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+', query_original) or \
       len(re.findall(r'\b[A-ZÁÉÍÓÚÑ][a-záéíóúñ]{2,}\b', query_original)) > 1 : # Más de una palabra capitalizada
        peso_bm25 = 0.65
        peso_emb = 0.35
        razon = ["Nombre Propio detectado -> Mayor peso BM25"]
        print(f"  INFO: {razon[0]}")
        return peso_bm25, peso_emb # Salida temprana si es muy específico

    # 2. Presencia de Años, Siglos o Fechas Específicas
    #    Regex mejorados para capturar varios formatos
    if re.search(r'\b\d{4}\b', query_lower) or \
       re.search(r'\bsiglo\s+(?:[xvií]+|[0-9]+)\b', query_lower) or \
       re.search(r'\b\d{1,2}(?:/| de |-| del )\w+(?:/| de |-| del )\d{2,4}\b', query_lower) or \
       re.search(r'\baño\s+\d{4}\b', query_lower):
        peso_bm25 = 0.7
        peso_emb = 0.3
        razon = ["Fecha/Año/Siglo detectado -> Mayor peso BM25"]
        print(f"  INFO: {razon[0]}")
        return peso_bm25, peso_emb # Salida temprana

    # 3. Presencia de Términos Técnicos o Acrónimos MUY específicos (si se conocen)
    #    Esta parte es dependiente del dominio, pero podemos buscar patrones comunes.
    #    Ej: Palabras en mayúsculas (ADN, ARN, SI, EBAU, ONU), términos con guiones/números (COVID-19, F-1)
    if re.search(r'\b[A-ZÁÉÍÓÚÑ]{2,}\b', query_original) or \
       re.search(r'\b\w+[-]\d+\b', query_lower):
       if not query_lower.isupper(): # Evitar que toda la query en mayúsculas active esto
            peso_bm25 = 0.6
            peso_emb = 0.4
            razon = ["Acrónimo/Término técnico probable detectado -> Mayor peso BM25"]
            print(f"  INFO: {razon[0]}")
            return peso_bm25, peso_emb # Salida temprana

    # 4. Búsqueda de Definiciones Literales ("define", "qué es exactamente", "cuál es la definición de")
    #    Son específicas, pero también conceptuales. Podríamos balancear.
    definicion_keywords = ["define", "definición de", "definir"]
    if any(keyword in query_lower for keyword in definicion_keywords) or query_lower.startswith("qué es"):
        peso_bm25 = 0.5
        peso_emb = 0.5
        razon = ["Petición de definición detectada -> Pesos equilibrados"]
        print(f"  INFO: {razon[0]}")
        return peso_bm25, peso_emb # Salida temprana

    # --- Indicadores de Conceptualidad (Favorecen Embeddings) ---

    # 5. Palabras Clave de Preguntas Conceptuales o de Proceso
    concept_keywords = ["cómo", "por qué", "explica", "describe", "relaciona", "compara",
                        "cuáles son las causas", "cuáles son las consecuencias", "cuál es la importancia",
                        "cuál es el significado", "proceso de", "evolución de", "impacto de",
                        "función de", "origen de", "características de", "tipos de"]
    if any(keyword in query_lower for keyword in concept_keywords):
        peso_bm25 = 0.3
        peso_emb = 0.7
        razon = ["Palabra clave conceptual detectada -> Mayor peso Embedding"]
        print(f"  INFO: {razon[0]}")
        return peso_bm25, peso_emb # Salida temprana

    # 6. Consultas más largas y descriptivas (heurística menos fiable, pero puede ayudar)
    #    Si no se activó nada específico antes, y la query es larga, probablemente sea conceptual.
    if len(query.split()) > 8: # Umbral ajustable
        peso_bm25 = 0.35
        peso_emb = 0.65
        razon = ["Consulta larga detectada -> Ligero sesgo Embedding"]
        # No retornamos aún, puede haber otros factores, pero ajustamos pesos

    # --- Decisión Final ---
    # Si ninguna regla fuerte se activó, mantenemos los pesos base o los ajustados por longitud
    print(f"  INFO: {razon[0]}")
    return peso_bm25, peso_emb
def norm_score(score, min_score, max_score):
    """Normaliza un score entre 0 y 1 usando Min-Max."""
    # Evitar división por cero si todos los scores son iguales
    if (max_score - min_score) == 0:
        return 0.0 if min_score == 0 else 0.5 # O alguna otra heurística, 0 si min es 0.
    return (score - min_score) / (max_score - min_score)


print("INFO: Celda 3 - Funciones auxiliares definidas (tokenizer, pesos, normalización).")
print(f"  - Usando tokenizer: {tokenizer_for_bm25.__name__}")
print("--- Fin Celda 3 ---")

INFO: Celda 3 - Funciones auxiliares definidas (tokenizer, pesos, normalización).
  - Usando tokenizer: simple_tokenizer
--- Fin Celda 3 ---


In [ ]:
# --- Celda 4: Definición de la Función Retriever Principal ---

def my_hybrid_rerank_retriever(query: str) -> str:
    """
    Función retriever que usa búsqueda híbrida (FAISS + BM25) con pesos dinámicos
    y reranking con CrossEncoder. Carga los recursos en la primera llamada.
    """
    global is_retriever_initialized, embeddings_model, faiss_index, texts, metadatas, bm25, reranker
    # Las variables globales se modifican aquí si es la primera vez

    # --- Bloque de Inicialización (se ejecuta solo la primera vez) ---
    if not is_retriever_initialized:
        print("INFO: Inicializando el retriever HÍBRIDO por primera vez...")
        try:
            # 1. Cargar modelo de Embedding Azure
            print("  Inicializando: 1. Cargando modelo Embedding Azure...")
            if not AZURE_EMBEDDING_DEPLOYMENT_NAME:
                raise ValueError("Variable AZURE_EMBEDDING_DEPLOYMENT_NAME no definida.")
            azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
            azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
            if not azure_endpoint or not azure_api_key:
                 raise ValueError("Credenciales AZURE_OPENAI_ENDPOINT o AZURE_OPENAI_API_KEY no encontradas.")

            embeddings_model = AzureOpenAIEmbeddings(
                azure_endpoint=azure_endpoint,
                openai_api_key=azure_api_key,
                azure_deployment=AZURE_EMBEDDING_DEPLOYMENT_NAME,
                openai_api_version=AZURE_API_VERSION,
            )
            print(f"     Modelo Embedding Azure ({AZURE_EMBEDDING_DEPLOYMENT_NAME}) cargado.")

            # 2. Cargar índice FAISS
            print("  Inicializando: 2. Cargando índice FAISS...")
            if not os.path.exists(INDEX_PATH):
                 raise FileNotFoundError(f"No se encontró el archivo de índice FAISS en: {INDEX_PATH}")
            faiss_index = faiss.read_index(INDEX_PATH)
            print(f"     Índice FAISS cargado desde '{INDEX_PATH}' ({faiss_index.ntotal} vectores).")

            # 3. Cargar textos y metadatos
            print("  Inicializando: 3. Cargando textos y metadatos...")
            if not os.path.exists(TEXTS_PATH): raise FileNotFoundError(f"Archivo no encontrado: {TEXTS_PATH}")
            if not os.path.exists(METAS_PATH): raise FileNotFoundError(f"Archivo no encontrado: {METAS_PATH}")
            with open(TEXTS_PATH, "rb") as f:
                texts = pickle.load(f)
            with open(METAS_PATH, "rb") as f:
                metadatas = pickle.load(f)
            print(f"     Textos ({len(texts)}) y Metadatos ({len(metadatas)}) cargados.")

            # 4. Verificación Crítica de Tamaños
            print("  Inicializando: 4. Verificando tamaños...")
            if not (faiss_index.ntotal == len(texts) == len(metadatas)):
                error_msg = f"¡ERROR CRÍTICO DE TAMAÑO! FAISS={faiss_index.ntotal}, Textos={len(texts)}, Metadatos={len(metadatas)}."
                print(error_msg)
                raise ValueError(error_msg)
            else:
                print("     OK: Tamaños coinciden.")

            # 5. Inicializar BM25
            print(f"  Inicializando: 5. Tokenizando documentos para BM25 ({tokenizer_for_bm25.__name__})...")
            # Asegurarse de que texts es una lista de strings
            if not isinstance(texts, list) or not all(isinstance(t, str) for t in texts):
                 raise TypeError("La variable 'texts' debe ser una lista de strings para BM25.")
            tokenized_docs = [tokenizer_for_bm25(txt) for txt in texts]
            bm25 = BM25Okapi(tokenized_docs)
            print("     Índice BM25 creado.")

            # 6. Inicializar Reranker (CrossEncoder)
            print(f"  Inicializando: 6. Cargando modelo Reranker '{RERANKER_MODEL}'...")
            reranker = CrossEncoder(RERANKER_MODEL)
            print("     Reranker cargado.")

            # 7. Marcar como inicializado
            is_retriever_initialized = True
            print("INFO: Inicialización del retriever HÍBRIDO completada.")

        except Exception as e:
            print(f"ERROR FATAL inicializando el retriever híbrido: {e}")
            traceback.print_exc()
            # Propagar el error para detener la ejecución si la inicialización falla
            raise RuntimeError("Fallo al inicializar el retriever híbrido.") from e
    # --- Fin Bloque de Inicialización ---

    # --- Bloque de Búsqueda Híbrida y Reranking ---
    print(f"\n--- (RAG Híbrido + Rerank) Buscando contexto para: '{query}' ---")
    if not is_retriever_initialized:
        # Esto no debería ocurrir si la inicialización se fuerza arriba, pero por seguridad:
        raise RuntimeError("El retriever no está inicializado. Hubo un error previo.")

    try:
        # 1. Obtener embedding de la consulta (usando Azure)
        print("  1. Obteniendo embedding Azure...")
        query_embedding = embeddings_model.embed_query(query)
        print("     Embedding obtenido.")

        # 2. Búsqueda FAISS (vectorial)
        print(f"  2. Realizando búsqueda FAISS (k={K_FAISS_INITIAL})...")
        distances, faiss_indices = faiss_index.search(np.array([query_embedding], dtype=np.float32), K_FAISS_INITIAL)
        # Usamos distancia L2, convertir a pseudo-similitud (mayor es mejor)
        # Cuidado: 1/(1+d) puede dar valores muy pequeños si d es grande.
        # Otra opción: max(0, 1 - distance / max_possible_distance_or_heuristic)
        # Mantenemos 1/(1+d) por simplicidad, pero vigila los rangos.
        faiss_sims = 1.0 / (1.0 + distances[0])
        # Crear dict {id: score} filtrando los -1 (índices inválidos)
        faiss_results = {idx: sim for idx, sim in zip(faiss_indices[0], faiss_sims) if idx != -1}
        print(f"     Búsqueda FAISS -> {len(faiss_results)} candidatos.")

        # 3. Búsqueda BM25 (palabras clave)
        print(f"  3. Realizando búsqueda BM25 (k={K_BM25_INITIAL})...")
        tokenized_query = tokenizer_for_bm25(query)
        # get_scores devuelve scores para *todos* los documentos
        all_bm25_scores = bm25.get_scores(tokenized_query)
        # Obtener los K mejores índices y scores de BM25
        bm25_top_indices = np.argsort(all_bm25_scores)[::-1][:K_BM25_INITIAL]
        # Crear dict {id: score} solo para los top K y con score > 0
        bm25_results = {idx: all_bm25_scores[idx] for idx in bm25_top_indices if all_bm25_scores[idx] > 0}
        print(f"     Búsqueda BM25 -> {len(bm25_results)} candidatos.")

        # 4. Fusión Híbrida con Pesos Dinámicos
        print("  4. Fusionando resultados...")
        peso_bm25, peso_emb = calcular_pesos_dinamicos(query)
        candidate_ids = set(faiss_results.keys()) | set(bm25_results.keys())
        print(f"     Total IDs candidatos únicos: {len(candidate_ids)}")

        # Normalizar scores ANTES de combinar
        faiss_scores_list = list(faiss_results.values())
        min_faiss, max_faiss = (min(faiss_scores_list), max(faiss_scores_list)) if faiss_scores_list else (0,0)
        bm25_scores_list = list(bm25_results.values())
        min_bm25, max_bm25 = (min(bm25_scores_list), max(bm25_scores_list)) if bm25_scores_list else (0,0)

        hybrid_scores = {}
        for idx in candidate_ids:
            score_f = faiss_results.get(idx, 0.0)
            score_b = bm25_results.get(idx, 0.0)
            # Normalizar y combinar
            norm_f = norm_score(score_f, min_faiss, max_faiss)
            norm_b = norm_score(score_b, min_bm25, max_bm25)
            hybrid_scores[idx] = (peso_emb * norm_f) + (peso_bm25 * norm_b)

        # Ordenar por score híbrido y seleccionar K para rerankear
        sorted_hybrid_ids = sorted(hybrid_scores, key=hybrid_scores.get, reverse=True)
        top_hybrid_candidates_ids = sorted_hybrid_ids[:K_RERANK]
        print(f"     {len(top_hybrid_candidates_ids)} candidatos seleccionados para reranking.")

        # 5. Reranking con CrossEncoder
        print(f"  5. Rerankeando con '{RERANKER_MODEL}'...")
        if not top_hybrid_candidates_ids:
             print("     No hay candidatos para rerankear.")
             reranked_docs_info = []
        else:
            # Crear pares (query, texto_del_chunk)
            # Asegurarse de que texts[doc_id] existe y es string
            rerank_pairs = []
            valid_ids_for_rerank = []
            for doc_id in top_hybrid_candidates_ids:
                 if 0 <= doc_id < len(texts) and isinstance(texts[doc_id], str):
                      rerank_pairs.append((query, texts[doc_id]))
                      valid_ids_for_rerank.append(doc_id)
                 else:
                      print(f"     ADVERTENCIA: Índice {doc_id} inválido o texto no es string, omitiendo para reranking.")

            if not rerank_pairs:
                 print("     No quedaron pares válidos para rerankear.")
                 reranked_docs_info = []
            else:
                 # Predecir scores con el reranker
                 reranker_scores = reranker.predict(rerank_pairs, show_progress_bar=False)

                 # Crear lista de resultados rerankeados con ID, texto, metadata y score
                 reranked_docs_info = []
                 for i, doc_id in enumerate(valid_ids_for_rerank):
                      # Asegurarse que metadatas[doc_id] existe
                      doc_metadata = metadatas[doc_id] if 0 <= doc_id < len(metadatas) else {}
                      reranked_docs_info.append({
                          "doc_id": doc_id,
                          "text": texts[doc_id],
                          "metadata": doc_metadata,
                          "reranker_score": float(reranker_scores[i])
                      })

                 # Ordenar finalmente por el score del reranker
                 reranked_docs_info.sort(key=lambda x: x["reranker_score"], reverse=True)
                 print(f"     Reranking completado.")

        # 6. Seleccionar los K_FINAL mejores y formatear contexto
        final_top_docs = reranked_docs_info[:K_FINAL]
        print(f"  6. Seleccionando los {len(final_top_docs)} mejores resultados finales.")

        # DEBUG: Mostrar los reranker scores de los top K_FINAL
        for i, doc_info in enumerate(final_top_docs):
            print(f"     Doc {i+1} Final (ID {doc_info['doc_id']}): Reranker Score = {doc_info['reranker_score']:.4f}")

        # Formatear contexto para el LLM
        context_parts = []
        for doc_info in final_top_docs:
             # Obtener fuente de metadatos, con fallback
             source = doc_info['metadata'].get('source', doc_info['metadata'].get('filename', f'ID:{doc_info["doc_id"]}'))
             context_parts.append(f"Fuente: {source} | Contenido: {doc_info['text']}")

        context = "\n\n---\n\n".join(context_parts)

        # Imprimir un preview del contexto final
        context_preview_length = 500
        print(f"--- Contexto Formateado Final ({len(final_top_docs)} chunks):\n{context[:context_preview_length]}...\n---")

        if not final_top_docs:
             print("ADVERTENCIA: No se recuperaron documentos relevantes tras el reranking.")
             # Devolver un mensaje claro en lugar de string vacío si no hay contexto
             return "No se encontró información relevante en el corpus para esta consulta."
        return context

    except Exception as e:
        print(f"ERROR durante la recuperación RAG Híbrida/Rerank: {e}")
        traceback.print_exc()
        # Devolver un mensaje de error claro
        return f"Se produjo un error durante la búsqueda de contexto: {e}"
    # --- Fin Bloque de Búsqueda ---


print("INFO: Celda 4 - Función principal 'my_hybrid_rerank_retriever' definida.")
print("--- Fin Celda 4 ---")

INFO: Celda 4 - Función principal 'my_hybrid_rerank_retriever' definida.
--- Fin Celda 4 ---


In [ ]:
# --- Celda 5: Asignación y Confirmación ---

# Asigna tu NUEVA función híbrida para ser usada por el resto de tu código/notebook
retriever_function = my_hybrid_rerank_retriever

print("INFO: Celda 5 - 'retriever_function' asignada a la implementación HÍBRIDA 'my_hybrid_rerank_retriever'.")
print("      El retriever (modelos, índices, etc.) se inicializará en la PRIMERA llamada a 'retriever_function'.")
print("--- Fin Celda 5 ---")

INFO: Celda 5 - 'retriever_function' asignada a la implementación HÍBRIDA 'my_hybrid_rerank_retriever'.
      El retriever (modelos, índices, etc.) se inicializará en la PRIMERA llamada a 'retriever_function'.
--- Fin Celda 5 ---


In [ ]:
# --- Celda 6: Prueba de Ejecución ---

print("\n--- Ejecutando prueba del retriever HÍBRIDO ---")
# Define una consulta de prueba
test_query_1 = "¿Cuáles son las diferencias entre las celulas eucariotas y procariotas?" # Específica
test_query_2 = "Explícame el origen de las células eucariotas" # Conceptual
test_query_3 = "Tipos de microscopios ópticos" # Muy específica

# Elige qué consulta probar:
query_a_probar = test_query_3

print(f"Prueba con consulta: '{query_a_probar}'")

try:
   # Primera llamada (realizará la inicialización si aún no se ha hecho)
   test_context = retriever_function(query_a_probar)

   print("\n--- Resultado de la Prueba ---")
   if "Error durante la recuperación" in test_context or "No se encontró información relevante" in test_context:
       print(f"Respuesta del Retriever (INFO/ERROR):\n{test_context}")
   else:
       # Imprime una parte del contexto recuperado
       print(f"Contexto recuperado (primeros 1000 caracteres):\n{test_context[:1000]}...")

   print("\n--- Prueba de Retriever Híbrida Finalizada (Revisar logs y contexto para calidad) ---")

   # Opcional: Segunda llamada para verificar que la inicialización no se repite
   # print("\n--- Ejecutando segunda llamada (debería ser más rápida) ---")
   # test_context_2 = retriever_function("¿Qué son los actos administrativos?")
   # print(f"Contexto segunda llamada (preview):\n{test_context_2[:200]}...")
   # print("--- Segunda llamada completada ---")


except Exception as e:
   print(f"\n--- ERROR CATASTRÓFICO en la prueba del retriever: {e} ---")
   traceback.print_exc()

print("--- Fin Celda 6 ---")


--- Ejecutando prueba del retriever HÍBRIDO ---
Prueba con consulta: 'Tipos de microscopios ópticos'
INFO: Inicializando el retriever HÍBRIDO por primera vez...
  Inicializando: 1. Cargando modelo Embedding Azure...
     Modelo Embedding Azure (text-embedding-3-small) cargado.
  Inicializando: 2. Cargando índice FAISS...


     Índice FAISS cargado desde 'educacion.index' (109 vectores).
  Inicializando: 3. Cargando textos y metadatos...
     Textos (109) y Metadatos (109) cargados.
  Inicializando: 4. Verificando tamaños...
     OK: Tamaños coinciden.
  Inicializando: 5. Tokenizando documentos para BM25 (simple_tokenizer)...
     Índice BM25 creado.
  Inicializando: 6. Cargando modelo Reranker 'cross-encoder/ms-marco-MiniLM-L-12-v2'...
     Reranker cargado.
INFO: Inicialización del retriever HÍBRIDO completada.

--- (RAG Híbrido + Rerank) Buscando contexto para: 'Tipos de microscopios ópticos' ---
  1. Obteniendo embedding Azure...
     Embedding obtenido.
  2. Realizando búsqueda FAISS (k=100)...
     Búsqueda FAISS -> 100 candidatos.
  3. Realizando búsqueda BM25 (k=100)...
     Búsqueda BM25 -> 100 candidatos.
  4. Fusionando resultados...
  INFO: Palabra clave conceptual detectada -> Mayor peso Embedding
     Total IDs candidatos únicos: 109
     100 candidatos seleccionados para reranking.
  5. Re

In [19]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_together import ChatTogether # O from langchain_fireworks import ChatFireworks
from langchain_community.llms import HuggingFaceHub
# Para simular RAG y LangChain Expression Language (LCEL)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

dotenv_path = '/home/azureuser/cloudfiles/code/Users/papastor/.env' # O '/ruta/completa/a/.env'
loaded = load_dotenv(dotenv_path=dotenv_path)
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm_providers = {}

# 1. Azure OpenAI (GPT-4 Turbo)
try:
    llm_providers['gpt4_turbo_azure'] = AzureChatOpenAI(
        openai_api_version="2024-02-15-preview", # O la versión API que uses
        azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        temperature=0.1 # Ajusta según necesites
    )
    print("INFO: AzureChatOpenAI (GPT-4 Turbo) cargado.")
except Exception as e:
    print(f"ERROR: No se pudo cargar AzureChatOpenAI: {e}")
    print("Verifica las variables de entorno AZURE_OPENAI...")

# 2. Google Gemini Pro 1.5
try:
    llm_providers['gemini_1.5_pro'] = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro-latest", # O el identificador específico que prefieras
        google_api_key=os.getenv("GOOGLE_API_KEY"),
        temperature=0.1,
        convert_system_message_to_human=True # Buena práctica para algunos modelos Gemini
    )
    print("INFO: ChatGoogleGenerativeAI (Gemini 1.5 Pro) cargado.")
except Exception as e:
    print(f"ERROR: No se pudo cargar ChatGoogleGenerativeAI: {e}")
    print("Verifica la variable de entorno GOOGLE_API_KEY.")

mixtral_repo_id = "mistralai/Mixtral-8x22B-Instruct-v0.1"
if hf_token:
    try:
        llm_providers['mixtral_8x22b_hf'] = HuggingFaceHub(
            repo_id=mixtral_repo_id,
            huggingfacehub_api_token=hf_token,
            model_kwargs={"temperature": 0.1, "max_new_tokens": 1024} # Ajusta parámetros
        )
        print(f"INFO: HuggingFaceHub ({mixtral_repo_id}) cargado.")
    except Exception as e:
        print(f"ERROR: No se pudo cargar Mixtral ({mixtral_repo_id}) via HuggingFaceHub: {e}")
        print("Verifica el repo_id y tu token de HF. El modelo puede no estar disponible en la Inference API gratuita/pública.")
else:
     print("INFO: Omitiendo carga de Mixtral desde HF por falta de token.")


# 4. Hugging Face Hub (Llama 3 70B)
# Verifica el ID exacto del repositorio. Requiere aceptar licencia de Meta.
llama3_repo_id = "meta-llama/Meta-Llama-3-70B-Instruct"
if hf_token:
    try:
        # Puede que necesites aceptar los términos de Llama 3 en Hugging Face primero
        llm_providers['llama3_70b_hf'] = HuggingFaceHub(
            repo_id=llama3_repo_id,
            huggingfacehub_api_token=hf_token,
            model_kwargs={"temperature": 0.1, "max_new_tokens": 1024} # Ajusta parámetros
        )
        print(f"INFO: HuggingFaceHub ({llama3_repo_id}) cargado.")
    except Exception as e:
        # Puede fallar si no has aceptado la licencia en HF o si el modelo no está en la API pública
        print(f"ERROR: No se pudo cargar Llama 3 ({llama3_repo_id}) via HuggingFaceHub: {e}")
        print("Verifica el repo_id, tu token de HF y asegúrate de haber aceptado los términos de uso de Llama 3 en Hugging Face. El modelo puede no estar disponible en la Inference API gratuita/pública.")
else:
    print("INFO: Omitiendo carga de Llama 3 desde HF por falta de token.")


print(f"\n--- LLMs disponibles para probar: {list(llm_providers.keys())} ---")


INFO: AzureChatOpenAI (GPT-4 Turbo) cargado.
INFO: ChatGoogleGenerativeAI (Gemini 1.5 Pro) cargado.


/tmp/ipykernel_4589/411009935.py:48: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm_providers['mixtral_8x22b_hf'] = HuggingFaceHub(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: HuggingFaceHub (mistralai/Mixtral-8x22B-Instruct-v0.1) cargado.
INFO: HuggingFaceHub (meta-llama/Meta-Llama-3-70B-Instruct) cargado.

--- LLMs disponibles para probar: ['gpt4_turbo_azure', 'gemini_1.5_pro', 'mixtral_8x22b_hf', 'llama3_70b_hf'] ---


In [ ]:
# --- Celda: Datos de Prueba (Adaptados para RAG) ---
import pandas as pd

# --- Tarea 1 (Adaptada): Resumen basado en RAG (Calidad + Refs) ---
# Lista de conceptos/temas CLAVE del PDF de Biología para resumir con referencias de página
topics_for_summary_bg = [
    "La Teoría de Oparin-Haldane sobre el origen químico de la vida y la formación del 'caldo nutritivo'.", # p. 7/11
    "Las principales propiedades físico-químicas del agua (polaridad, cohesividad, solubilidad, calor específico) y su importancia biológica según el texto.", # p. 5/35 a 7/35
    "El modelo de la doble hélice del ADN propuesto por Watson y Crick, destacando sus características estructurales clave.", # p. 35/35
    "El modelo de mosaico fluido de la membrana plasmática, incluyendo sus componentes lipídicos y proteicos.", # p. 4/24 y 5/24
    "El proceso y las causas de la desnaturalización de las proteínas, y su efecto en la funcionalidad.", # p. 29/35
    "Las etapas involucradas en la preparación de muestras para microscopía óptica.", # p. 5/11
]

# --- Tarea 2 (Adaptada): Generación de Preguntas basada en RAG (Calidad + Refs) ---
# Lista de conceptos/temas CLAVE del PDF para generar preguntas significativas con referencias de página
topics_for_question_gen_bg = [
    "Las etapas propuestas para la evolución química prebiótica (síntesis monómeros, polimerización, coacervatos).", # p. 8/11 y 9/11
    "Los cuatro niveles de estructura de las proteínas (primaria, secundaria, terciaria, cuaternaria) y qué tipo de enlaces estabiliza cada uno.", # p. 24/35 a 27/35
    "Los diferentes mecanismos de transporte pasivo a través de la membrana (difusión simple y facilitada por canal/transportador).", # p. 6/24 y 7/24
    "La Teoría Endosimbiótica/Simbiogénesis sobre el origen de mitocondrias y cloroplastos.", # Mencionado en p. 11/11 (ribosomas), p. 18/24 (plastos), p. 19/24 (mitocondrias), p. 11/11 (final - simbiogénesis)
    "Las diferencias funcionales y estructurales entre el Retículo Endoplasmático Rugoso (RER) y el Liso (REL).", # p. 11/24 y 12/24
    "La composición y estructura básica de un nucleosoma como unidad fundamental de la cromatina.", # p. 21/24
]

# --- Tarea 3: Preguntas de Contenido Académico (RAG - Calidad + Refs) ---
# Preguntas directas que requieren RAG sobre detalles específicos o explicaciones del PDF de Biología con referencias de página
rag_questions_bg = [
    "Según el experimento de Miller descrito en el texto, ¿qué tipo de moléculas orgánicas fundamentales para la vida se formaron?", # p. 8/11 (aminoácidos)
    "¿Cómo explica el texto la insolubilidad de los compuestos orgánicos no polares (hidrófobos) en agua?", # p. 7/35 (interrumpen estructura del agua, no forman enlaces H)
    "Describe la estructura 'collar de perlas' (10 nm) de la cromatina, mencionando sus componentes principales (ADN y ocho moléculas de histonas).", # p. 21/24
    "¿Cuál es la función específica de la enzima catalasa dentro de los peroxisomas, según se indica?", # p. 14/24 (destruir H2O2)
    "Explica la diferencia entre heterofagia y autofagia en el contexto de la función de los lisosomas.", # p. 13/24
    "¿Qué son los plasmodesmos y cuál es su función en relación a la pared celular vegetal?", # p. 10/24 (conexiones citoplasmáticas entre células contiguas)
]

# --- Crear DataFrames para organizar tareas ---
summary_tasks_df_bg = pd.DataFrame(topics_for_summary_bg, columns=['topic'])
question_gen_tasks_df_bg = pd.DataFrame(topics_for_question_gen_bg, columns=['topic'])
rag_qa_tasks_df_bg = pd.DataFrame(rag_questions_bg, columns=['question'])

print("Datos de prueba de ALTA CALIDAD con REF. PÁGINA (basados en contenido específico del PDF de Biología para RAG) preparados:")
print(f" - {len(summary_tasks_df_bg)} conceptos para resumir vía RAG.")
print(f" - {len(question_gen_tasks_df_bg)} conceptos para generar preguntas vía RAG.")
print(f" - {len(rag_qa_tasks_df_bg)} preguntas directas para RAG Q&A.")



Datos de prueba de ALTA CALIDAD con REF. PÁGINA (basados en contenido específico del PDF de Biología para RAG) preparados:
 - 6 conceptos para resumir vía RAG.
 - 6 conceptos para generar preguntas vía RAG.
 - 6 preguntas directas para RAG Q&A.


In [ ]:
# --- Celda: Datos de Prueba (Adaptados para RAG - Física 1 - Calidad + Refs) ---
import pandas as pd

# --- Tarea 1 (Adaptada): Resumen basado en RAG (Calidad + Refs - Física 1) ---
# Lista de conceptos/temas CLAVE del PDF de Física 1 para resumir
topics_for_summary_f = [
    "La definición y distinción entre magnitudes escalares y vectoriales.", # p. 7/19
    "La descripción del vector de posición, velocidad instantánea y aceleración instantánea en cinemática.", # p. 33/45 - 34/46
    "El enunciado y significado físico de las tres Leyes de Newton.", # p. 41/53, 43/55, 44/56
    "El concepto de trabajo realizado por una fuerza y su cálculo para una fuerza constante.", # p. 62/74 - 63/75
    "La definición de energía potencial asociada a fuerzas conservativas y la relación Wc = -ΔU.", # p. 67/79 - 68/80
    "La descripción del Movimiento Armónico Simple (MAS) a partir de la ecuación diferencial x¨ = -ω₀²x.", # p. 80/92
]

# --- Tarea 2 (Adaptada): Generación de Preguntas basada en RAG (Calidad + Refs - Física 1) ---
# Lista de conceptos/temas CLAVE del PDF para generar preguntas significativas
topics_for_question_gen_f = [
    "Las propiedades de las operaciones con vectores: suma, producto escalar y producto vectorial.", # p. 15/27 - 17/29
    "La diferencia entre sistemas de referencia inerciales y no inerciales según la primera ley de Newton.", # p. 42/54
    "La conservación del momento lineal y su relación con la ausencia de fuerzas externas netas.", # p. 53/65
    "La relación entre Trabajo y Energía Cinética (Teorema Trabajo-Energía).", # p. 65/77 - 66/78
    "El análisis del movimiento utilizando curvas de energía potencial (puntos de equilibrio estable/inestable, puntos de retorno).", # p. 72/84 - 73/85
    "Los diferentes regímenes de oscilaciones amortiguadas (subamortiguado, crítico, sobreamortiguado) y las características de sus soluciones.", # p. 95/107 - 96/108
]

# --- Tarea 3: Preguntas de Contenido Académico (RAG - Calidad + Refs - Física 1) ---
# Preguntas directas que requieren RAG sobre detalles específicos o explicaciones del PDF de Física 1
rag_questions_f = [
    "¿Cómo se define el producto escalar de dos vectores a y b (a · b) en términos de sus componentes cartesianas?", # p. 16/28
    "¿Qué son los 'cosenos directores' de un vector de posición en coordenadas cartesianas?", # p. 13/25
    "¿Cuál es la expresión matemática que define la Segunda Ley de Newton en su forma más general, involucrando el momento lineal (p)?", # p. 43/55 (F = d(mv)/dt o F=dp/dt)
    "¿Qué es el impulso lineal (I) de una fuerza F y cómo se relaciona con el cambio en el momento lineal (Δp)?", # p. 54/66 (I = ∫F dt = Δp)
    "Define la potencia instantánea (P) en términos del trabajo (dW) o de la fuerza (F) y la velocidad (v).", # p. 65/77 (P = dW/dt = F·v)
    "¿Cuál es la relación fundamental entre la fuerza conservativa Fx y la energía potencial U(x) en una dimensión?", # p. 69/81 (Fx = -dU/dx)
]

# --- Crear DataFrames para organizar tareas ---
summary_tasks_df_f = pd.DataFrame(topics_for_summary_f, columns=['topic'])
question_gen_tasks_df_f = pd.DataFrame(topics_for_question_gen_f, columns=['topic'])
rag_qa_tasks_df_f = pd.DataFrame(rag_questions_f, columns=['question'])

print("Datos de prueba de ALTA CALIDAD con REF. PÁGINA (basados en contenido específico del PDF de Física 1 para RAG) preparados:")
print(f" - {len(summary_tasks_df_f)} conceptos para resumir vía RAG.")
print(f" - {len(question_gen_tasks_df_f)} conceptos para generar preguntas vía RAG.")
print(f" - {len(rag_qa_tasks_df_f)} preguntas directas para RAG Q&A.")

# !!! IMPORTANTE: Necesitas tu función de retriever REAL aquí !!!
# Esta función debe buscar en el contenido INDEXADO del PDF de Física 1 completo.
# Ejemplo: from my_rag_module import fisica1_arquitectura_retriever
# retriever_function = fisica1_arquitectura_retriever

Datos de prueba de ALTA CALIDAD con REF. PÁGINA (basados en contenido específico del PDF de Física 1 para RAG) preparados:
 - 6 conceptos para resumir vía RAG.
 - 6 conceptos para generar preguntas vía RAG.
 - 6 preguntas directas para RAG Q&A.


In [ ]:
# --- Celda: Datos de Prueba (Adaptados para RAG - Historia España - Calidad + Refs) ---
import pandas as pd

# --- Tarea 1 (Adaptada): Resumen basado en RAG (Calidad + Refs - Historia España) ---
# Lista de conceptos/temas CLAVE del PDF de Historia de España para resumir
topics_for_summary_h = [
    "El proceso de hominización en la Península Ibérica, mencionando las principales especies y yacimientos.", # p. 8-9
    "El proceso de Romanización en Hispania: factores que lo favorecieron y principales manifestaciones (lengua, derecho, urbanismo, etc.).", # p. 13-15
    "La unificación territorial, política, religiosa y jurídica lograda durante la Monarquía Visigoda.", # p. 17
    "Las causas, desarrollo y consecuencias principales de la revuelta de los Comuneros en Castilla (1520-1522).", # p. 43
    "Los Decretos de Nueva Planta: objetivo, reinos afectados y consecuencias en la centralización administrativa y uniformización jurídica.", # p. 54-55
    "Principios fundamentales y características de la Constitución de 1812.", # p. 79-80
]

# --- Tarea 2 (Adaptada): Generación de Preguntas basada en RAG (Calidad + Refs - Historia España) ---
# Lista de conceptos/temas CLAVE del PDF para generar preguntas significativas
topics_for_question_gen_h = [
    "La organización política, económica y social de Al-Ándalus durante el Emirato y el Califato.", # p. 17-22 (especialmente p. 21-22)
    "Los diferentes tipos de repoblación cristiana (presura, concejil, Órdenes Militares/donadíos, repartimientos) y sus características.", # p. 28
    "La política exterior de Carlos I (V): principales conflictos con Francia, los Turcos y los príncipes alemanes.", # p. 43-44
    "Las reformas impulsadas durante el Despotismo Ilustrado de Carlos III en los ámbitos cultural, urbanístico y económico.", # p. 57-58
    "Las distintas tendencias políticas presentes en las Cortes de Cádiz (Liberales, Jovellanistas, Absolutistas).", # p. 78
    "Las causas que provocaron la independencia de las colonias americanas, según se detallan en el texto.", # p. 86
]

# --- Tarea 3: Preguntas de Contenido Académico (RAG - Calidad + Refs - Historia España) ---
# Preguntas directas que requieren RAG sobre detalles específicos o explicaciones del PDF de Historia de España
rag_questions_h = [
    "¿Qué culturas o pueblos colonizadores mediterráneos se mencionan como introductores de novedades (cultivos, torno, moneda, alfabeto) en la Península?", # p. 11 (Griegos y Fenicios)
    "¿Qué fue el Liber Iudiciorum (Fuero Juzgo) y qué importancia tuvo en la unificación visigoda?", # p. 17 (Unificación jurídica bajo Recesvinto)
    "Según la Concordia de Segovia (1475), ¿cómo se establecía el gobierno conjunto de Isabel y Fernando en Castilla y Aragón?", # p. 32, 36 (Gobierno conjunto pero reinos separados)
    "¿Qué consecuencias tuvo para España la Paz de Westfalia de 1648 mencionada en el contexto de Felipe IV?", # p. 46-47 (Reconocimiento independencia Holanda)
    "¿Qué estipulaba el Tratado de Fontainebleau de 1807 entre España y Francia respecto a Portugal?", # p. 71 (Permitir paso ejército francés, dividir Portugal)
    "¿Qué fue la Desamortización de Mendizábal y cuáles eran sus objetivos principales?", # p. 95, 107 (Venta bienes clero regular para pagar deuda/guerra carlista, crear grupo social liberal)
]

# --- Crear DataFrames para organizar tareas ---
summary_tasks_df_h = pd.DataFrame(topics_for_summary_h, columns=['topic'])
question_gen_tasks_df_h = pd.DataFrame(topics_for_question_gen_h, columns=['topic'])
rag_qa_tasks_df_h = pd.DataFrame(rag_questions_h, columns=['question'])

print("Datos de prueba de ALTA CALIDAD con REF. PÁGINA (basados en contenido específico del PDF de Historia España para RAG) preparados:")
print(f" - {len(summary_tasks_df_h)} conceptos para resumir vía RAG.")
print(f" - {len(question_gen_tasks_df_h)} conceptos para generar preguntas vía RAG.")
print(f" - {len(rag_qa_tasks_df_h)} preguntas directas para RAG Q&A.")

Datos de prueba de ALTA CALIDAD con REF. PÁGINA (basados en contenido específico del PDF de Historia España para RAG) preparados:
 - 6 conceptos para resumir vía RAG.
 - 6 conceptos para generar preguntas vía RAG.
 - 6 preguntas directas para RAG Q&A.


In [ ]:
# --- Celda: Datos de Prueba (Adaptados para RAG - Lengua y Lit. - Calidad + Refs) ---
import pandas as pd

# --- Tarea 1 (Adaptada): Resumen basado en RAG (Calidad + Refs - Lengua y Lit.) ---
# Lista de conceptos/temas CLAVE del PDF de Lengua y Lit. para resumir
topics_for_summary_l = [
    "Las propiedades del texto: coherencia, cohesión y adecuación, según se explican en el documento.", # p. 14, 15, 32, 45
    "La diferencia entre denotación y connotación y su relevancia en el lenguaje.", # p. 11
    "El concepto y tipos de deixis (personal, temporal, espacial; textual y contextual).", # p. 34-35
    "Las reglas generales de acentuación para palabras agudas, llanas y esdrújulas.", # p. 42
    "Descripción de las figuras literarias de repetición de palabras (anáfora, anadiplosis, concatenación, epanadiplosis).", # p. 93
    "Características principales del movimiento literario del Romanticismo presentadas en el texto.", # p. 122 (Sección Ilustración s.XVIII) - Aunque está en la p.123 el título, la descripción empieza en la 122
]

# --- Tarea 2 (Adaptada): Generación de Preguntas basada en RAG (Calidad + Refs - Lengua y Lit.) ---
# Lista de conceptos/temas CLAVE del PDF para generar preguntas significativas
topics_for_question_gen_l = [
    "Los elementos de la comunicación (emisor, receptor, mensaje, canal, código, contexto) y las funciones de la lengua asociadas a cada uno.", # p. 6-7
    "La clasificación de los verbos según su forma (simples, compuestos, perífrasis) y según su conjugación (regulares, irregulares).", # p. 64-65
    "Los diferentes tipos de estrofas según el número de versos (pareado, terceto, cuarteto, redondilla, etc.).", # p. 104-112
    "Los fenómenos que afectan a la medición de sílabas en métrica española (sinalefa, sinéresis, hiato, diéresis).", # p. 96
    "Los distintos valores y funciones gramaticales del pronombre 'SE'.", # p. 59
    "Las características que definen la lengua literaria y la diferencian del uso normal del lenguaje (Función Poética, Connotación, Plurisignificación, Ficcionalidad).", # p. 91
]

# --- Tarea 3: Preguntas de Contenido Académico (RAG - Calidad + Refs - Lengua y Lit.) ---
# Preguntas directas que requieren RAG sobre detalles específicos o explicaciones del PDF de Lengua y Lit.
rag_questions_l = [
    "¿Cuál es la diferencia principal entre lengua, habla y norma según las definiciones dadas?", # p. 5
    "Define qué es el 'tema' y el 'rema' en la organización de la información textual.", # p. 15
    "Explica el concepto de 'elipsis' como mecanismo de cohesión textual.", # p. 33
    "¿Cuándo lleva tilde diacrítica el monosílabo 'tú' y cuándo no?", # p. 43
    "¿Qué es un verso de 'arte mayor' y a partir de qué número de sílabas se considera como tal?", # p. 97
    "Describe la figura literaria 'Hipérbaton' y proporciona el ejemplo que aparece en el texto.", # p. 94
]

# --- Crear DataFrames para organizar tareas ---
summary_tasks_df_l = pd.DataFrame(topics_for_summary_l, columns=['topic'])
question_gen_tasks_df_l = pd.DataFrame(topics_for_question_gen_l, columns=['topic'])
rag_qa_tasks_df_l = pd.DataFrame(rag_questions_l, columns=['question'])

print("Datos de prueba de ALTA CALIDAD con REF. PÁGINA (basados en contenido específico del PDF de Lengua y Lit. para RAG) preparados:")
print(f" - {len(summary_tasks_df_l)} conceptos para resumir vía RAG.")
print(f" - {len(question_gen_tasks_df_l)} conceptos para generar preguntas vía RAG.")
print(f" - {len(rag_qa_tasks_df_l)} preguntas directas para RAG Q&A.")

# !!! IMPORTANTE: Necesitas tu función de retriever REAL aquí !!!
# Esta función debe buscar en el contenido INDEXADO del PDF de Lengua y Lit. completo.
# Ejemplo: from my_rag_module import lengua_lit_secundaria_retriever
# retriever_function = lengua_lit_secundaria_retriever

Datos de prueba de ALTA CALIDAD con REF. PÁGINA (basados en contenido específico del PDF de Lengua y Lit. para RAG) preparados:
 - 6 conceptos para resumir vía RAG.
 - 6 conceptos para generar preguntas vía RAG.
 - 6 preguntas directas para RAG Q&A.


In [ ]:
# --- Celda: Prompts y Función de Ejecución RAG Universal ---
import time
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# --- Plantillas de Prompt (Adaptadas para RAG) ---

# Prompt para Resumen basado en RAG
rag_summary_prompt_template = ChatPromptTemplate.from_template(
    """Contexto recuperado sobre el tema '{topic}':
    {context}

    Basándote ESTRICTAMENTE en el contexto proporcionado, genera un resumen conciso (aproximadamente 2-4 frases) sobre '{topic}' que capture las ideas clave presentadas en el contexto. Mantén un estilo académico y formal. Si el contexto no es suficiente o relevante, indícalo.

    Resumen Conciso:"""
)

# Prompt para Generación de Preguntas basado en RAG
rag_question_gen_prompt_template = ChatPromptTemplate.from_template(
    """Contexto recuperado sobre el tema '{topic}':
    {context}

    Basándote ESTRICTAMENTE en la información del contexto proporcionado sobre '{topic}', genera 1 pregunta tipo test (con 4 opciones, marca la correcta con '*') y 1 pregunta de desarrollo que evaluarían la comprensión de este tema específico según el contexto. Si el contexto no es suficiente o relevante, indícalo.

    Preguntas Generadas:"""
)

# Prompt para Q&A RAG (sin cambios, pero lo incluimos por completitud)
rag_qa_prompt_template = ChatPromptTemplate.from_template(
    """Contexto recuperado:
    {context}

    Basándote ESTRICTAMENTE en el contexto proporcionado, responde a la siguiente pregunta académica de forma clara y precisa.
    Si la respuesta no se encuentra en el contexto, indica que no puedes responder con la información dada.

    Pregunta: {question}

    Respuesta Precisa:"""
)

# --- Función Universal para Tareas Basadas en RAG ---
def run_rag_based_task(llm, user_query: str, task_prompt_template: ChatPromptTemplate, retriever_func, task_specific_input: dict):
    """
    Ejecuta una tarea completa basada en RAG (retrieve + generate).

    Args:
        llm: El cliente LLM de LangChain.
        user_query: La consulta original del usuario (concepto, pregunta). Usada para el retriever.
        task_prompt_template: La plantilla de prompt para la tarea específica (resumen, QG, Q&A).
        retriever_func: La función que realiza la búsqueda RAG. Debe devolver el contexto como un string
                        o una lista de objetos Document de LangChain.
        task_specific_input: Dict con datos adicionales para el prompt (ej: {'topic': 'X'} o {'question': 'Y'}).

    Returns:
        Tuple: (retrieved_context_str, response, retrieval_duration, llm_duration, error)
    """
    retrieved_context_str = ""
    response = ""
    retrieval_duration = 0.0
    llm_duration = 0.0
    error = None

    # 1. Recuperación
    start_time_retrieval = time.time()
    try:
        print(f"--- Retrieving context for query: '{user_query}'")
        retrieved_data = retriever_func(user_query) # Puede devolver str o List[Document]

        # Asegurarse de que el contexto sea un string para el prompt
        if isinstance(retrieved_data, list) and all(isinstance(doc, Document) for doc in retrieved_data):
             # Formato común si el retriever devuelve Documentos LangChain
            retrieved_context_str = "\n\n".join([doc.page_content for doc in retrieved_data])
            print(f"--- Retrieved {len(retrieved_data)} documents.")
        elif isinstance(retrieved_data, str):
            retrieved_context_str = retrieved_data # El retriever ya devolvió un string
            print("--- Retrieved context as a single string.")
        else:
            # Intentar convertir a string, o manejar como error si no es esperado
            print(f"--- WARNING: Unexpected retriever output type: {type(retrieved_data)}. Attempting str conversion.")
            retrieved_context_str = str(retrieved_data)

        print(f"--- Context Retrieved (first 500 chars): ---\n{retrieved_context_str[:500]}...\n-----------------------------------------")
        retrieval_duration = time.time() - start_time_retrieval

    except Exception as e:
        retrieval_duration = time.time() - start_time_retrieval
        print(f"ERROR during context retrieval for '{user_query}': {e}")
        traceback.print_exc() # Imprime el traceback completo
        retrieved_context_str = f"Error retrieving context: {e}"
        # Considerar si continuar o devolver error aquí mismo
        # return retrieved_context_str, None, retrieval_duration, 0.0, str(e)


    # 2. Generación (usando LCEL para pasar contexto y datos específicos)
    try:
        # *** INICIO DE LA CORRECCIÓN ***
        # Prepara los argumentos para assign. Cada valor debe ser un callable.
        # Usamos un argumento por defecto en el lambda interno para capturar
        # correctamente el valor de 'value' en cada iteración.
        assign_args = {
            "context": lambda x: retrieved_context_str, # Pasa el contexto recuperado
            **{key: (lambda value_copy=value: lambda x: value_copy)()
               for key, value in task_specific_input.items()} # Pasa los valores estáticos como callables
        }
        # *** FIN DE LA CORRECCIÓN ***

        print(f"--- Generating response with LLM. Prompt inputs expected: {task_prompt_template.input_variables}. Provided via assign: {list(assign_args.keys())}")

        rag_chain = (
            RunnablePassthrough.assign(**assign_args)
            | task_prompt_template
            | llm
            | StrOutputParser()
        )

        start_time_llm = time.time()
        # Invocamos la cadena. Un diccionario vacío es suficiente como input inicial
        # ya que 'assign_args' inyecta todo lo necesario para el prompt.
        response = rag_chain.invoke({})
        llm_duration = time.time() - start_time_llm
        print(f"--- LLM Response Generated (first 500 chars): ---\n{str(response)[:500]}...\n-----------------------------------------")


    except Exception as e:
        llm_duration = time.time() - start_time_llm if 'start_time_llm' in locals() else 0.0
        error_vars = task_prompt_template.input_variables if hasattr(task_prompt_template, 'input_variables') else 'N/A'
        print(f"ERROR during RAG generation (expected prompt inputs: {error_vars}): {e}")
        traceback.print_exc() # Imprime el traceback completo
        response = None # Asegurarse de que response es None en caso de error
        error = str(e)

    return retrieved_context_str, response, retrieval_duration, llm_duration, error

print("Prompts adaptados y función RAG universal (CORREGIDA) definidos.")

Prompts adaptados y función RAG universal (CORREGIDA) definidos.


In [ ]:
# --- Celda: Ejecución de Pruebas (Adaptada para RAG Universal) ---
results_list = [] # Lista para almacenar todos los resultados

# --- Ejecutar Tarea 1: Resumen RAG ---
print("\n--- Iniciando Tarea 1: Resumen basado en RAG ---")
for index, row in summary_tasks_df_bg.iterrows():
    topic = row['topic']
    # La consulta al retriever puede ser el mismo topic o algo más elaborado
    retriever_query = f"Información detallada sobre {topic}" # O simplemente 'topic'
    for llm_name, llm_client in llm_providers.items():
        print(f"Resumiendo '{topic}' con {llm_name}...")
        context, response, retrieval_duration, llm_duration, error = run_rag_based_task(
            llm=llm_client,
            user_query=retriever_query, # Query para buscar en RAG
            task_prompt_template=rag_summary_prompt_template, # Prompt de resumen
            retriever_func=retriever_function, # Tu retriever
            task_specific_input={"topic": topic} # Input específico para el prompt de resumen
        )
        results_list.append({
            "task_type": "RAG Summarization",
            "input_id": topic,
            "llm_provider": llm_name,
            "input_data": retriever_query, # Guardamos qué se usó para buscar
            "output_response": response,
            "retrieved_context": context, # Crucial guardar el contexto
            "duration_seconds": llm_duration,
            "retrieval_duration_seconds": retrieval_duration,
            "error": error
        })
        if "azure" in llm_name.lower():
             # El error 429 pedía esperar 60 segundos, ponemos un poco más por seguridad.
             pause_duration = 65
             print(f"--- Pausando por {pause_duration} segundos después de la llamada a {llm_name} para evitar rate limits... ---")
             time.sleep(pause_duration)

# --- Ejecutar Tarea 2: Generación de Preguntas RAG ---
print("\n--- Iniciando Tarea 2: Generación de Preguntas basada en RAG ---")
for index, row in question_gen_tasks_df_bg.iterrows():
    topic = row['topic']
    retriever_query = f"Detalles clave sobre {topic}" # O simplemente 'topic'
    for llm_name, llm_client in llm_providers.items():
        print(f"Generando preguntas sobre '{topic}' con {llm_name}...")
        context, response, retrieval_duration, llm_duration, error = run_rag_based_task(
            llm=llm_client,
            user_query=retriever_query,
            task_prompt_template=rag_question_gen_prompt_template, # Prompt de gen. preguntas
            retriever_func=retriever_function,
            task_specific_input={"topic": topic} # Input específico para el prompt
        )
        results_list.append({
            "task_type": "RAG Question Generation",
            "input_id": topic,
            "llm_provider": llm_name,
            "input_data": retriever_query,
            "output_response": response,
            "retrieved_context": context,
            "duration_seconds": llm_duration,
            "retrieval_duration_seconds": retrieval_duration,
            "error": error
        })
        if "azure" in llm_name.lower():
             # El error 429 pedía esperar 60 segundos, ponemos un poco más por seguridad.
             pause_duration = 65
             print(f"--- Pausando por {pause_duration} segundos después de la llamada a {llm_name} para evitar rate limits... ---")
             time.sleep(pause_duration)

# --- Ejecutar Tarea 3: Preguntas RAG Q&A ---
print("\n--- Iniciando Tarea 3: Preguntas Directas RAG Q&A ---")
for index, row in rag_qa_tasks_df_bg.iterrows():
    question = row['question']
    # Para Q&A, la query al retriever suele ser la misma pregunta
    retriever_query = question
    for llm_name, llm_client in llm_providers.items():
        print(f"Respondiendo pregunta RAG '{question[:50]}...' con {llm_name}...")
        context, response, retrieval_duration, llm_duration, error = run_rag_based_task(
            llm=llm_client,
            user_query=retriever_query,
            task_prompt_template=rag_qa_prompt_template, # Prompt de Q&A
            retriever_func=retriever_function,
            task_specific_input={"question": question} # Input específico para el prompt
        )
        results_list.append({
            "task_type": "RAG Q&A",
            "input_id": question,
            "llm_provider": llm_name,
            "input_data": retriever_query,
            "output_response": response,
            "retrieved_context": context,
            "duration_seconds": llm_duration,
            "retrieval_duration_seconds": retrieval_duration,
            "error": error
        })
        if "azure" in llm_name.lower():
             # El error 429 pedía esperar 60 segundos, ponemos un poco más por seguridad.
             pause_duration = 65
             print(f"--- Pausando por {pause_duration} segundos después de la llamada a {llm_name} para evitar rate limits... ---")
             time.sleep(pause_duration)

# --- Convertir resultados a DataFrame ---
results_df = pd.DataFrame(results_list)
print(f"\n--- Pruebas RAG completadas. {len(results_df)} resultados generados. ---")
display(results_df.head())

# --- Guardar Resultados (sin cambios) ---
results_df.to_csv("llm_rag_evaluation_results.csv", index=False)
print("\nResultados RAG completos guardados en 'llm_rag_evaluation_results.csv'")


--- Iniciando Tarea 1: Resumen basado en RAG ---
Resumiendo 'La Teoría de Oparin-Haldane sobre el origen químico de la vida y la formación del 'caldo nutritivo'.' con gpt4_turbo_azure...
--- Retrieving context for query: 'Información detallada sobre La Teoría de Oparin-Haldane sobre el origen químico de la vida y la formación del 'caldo nutritivo'.'

--- (RAG Híbrido + Rerank) Buscando contexto para: 'Información detallada sobre La Teoría de Oparin-Haldane sobre el origen químico de la vida y la formación del 'caldo nutritivo'.' ---
  1. Obteniendo embedding Azure...
     Embedding obtenido.
  2. Realizando búsqueda FAISS (k=100)...
     Búsqueda FAISS -> 100 candidatos.
  3. Realizando búsqueda BM25 (k=100)...
     Búsqueda BM25 -> 100 candidatos.
  4. Fusionando resultados...
  INFO: Nombre Propio detectado -> Mayor peso BM25
     Total IDs candidatos únicos: 107
     100 candidatos seleccionados para reranking.
  5. Rerankeando con 'cross-encoder/ms-marco-MiniLM-L-12-v2'...
     Re

Traceback (most recent call last):
  File "/tmp/ipykernel_4795/2384842375.py", line 120, in run_rag_based_task
    response = rag_chain.invoke({})
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 3034, in invoke
    input = context.run(step.invoke, input, config)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/language_models/chat_models.py", line 369, in invoke
    self.generate_prompt(
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/language_models/chat_models.py", line 946, in generate_prompt
    return self.generate(prompt_messages, stop=stop, callbacks=callbacks, **kwargs)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/language_models/chat_models.py", line 765, in generate
    self._generate_with_cache(
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/language_model

Resumiendo 'La Teoría de Oparin-Haldane sobre el origen químico de la vida y la formación del 'caldo nutritivo'.' con gemini_1.5_pro...
--- Retrieving context for query: 'Información detallada sobre La Teoría de Oparin-Haldane sobre el origen químico de la vida y la formación del 'caldo nutritivo'.'

--- (RAG Híbrido + Rerank) Buscando contexto para: 'Información detallada sobre La Teoría de Oparin-Haldane sobre el origen químico de la vida y la formación del 'caldo nutritivo'.' ---
  1. Obteniendo embedding Azure...
     Embedding obtenido.
  2. Realizando búsqueda FAISS (k=100)...
     Búsqueda FAISS -> 100 candidatos.
  3. Realizando búsqueda BM25 (k=100)...
     Búsqueda BM25 -> 100 candidatos.
  4. Fusionando resultados...
  INFO: Nombre Propio detectado -> Mayor peso BM25
     Total IDs candidatos únicos: 107
     100 candidatos seleccionados para reranking.
  5. Rerankeando con 'cross-encoder/ms-marco-MiniLM-L-12-v2'...
     Reranking completado.
  6. Seleccionando los 3 mejores

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:388: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


--- LLM Response Generated (first 500 chars): ---
La teoría de Oparin-Haldane propone que la vida se originó en una atmósfera primitiva reductora (CH₄, NH₃, H₂, H₂O) donde la energía, como las descargas eléctricas, permitió la formación de moléculas orgánicas en un "caldo nutritivo".  Estas moléculas se asociaron y eventualmente adquirieron la capacidad de autorreplicarse, marcando el inicio de la vida en un momento concreto con condiciones irrepetibles en la actualidad debido a la presencia de oxígeno y seres vivos....
-----------------------------------------
Resumiendo 'La Teoría de Oparin-Haldane sobre el origen químico de la vida y la formación del 'caldo nutritivo'.' con mixtral_8x22b_hf...
--- Retrieving context for query: 'Información detallada sobre La Teoría de Oparin-Haldane sobre el origen químico de la vida y la formación del 'caldo nutritivo'.'

--- (RAG Híbrido + Rerank) Buscando contexto para: 'Información detallada sobre La Teoría de Oparin-Haldane sobre el origen quím

...

     Embedding obtenido.
  2. Realizando búsqueda FAISS (k=100)...
     Búsqueda FAISS -> 100 candidatos.
  3. Realizando búsqueda BM25 (k=100)...
     Búsqueda BM25 -> 100 candidatos.
  4. Fusionando resultados...
  INFO: Nombre Propio detectado -> Mayor peso BM25
     Total IDs candidatos únicos: 106
     100 candidatos seleccionados para reranking.
  5. Rerankeando con 'cross-encoder/ms-marco-MiniLM-L-12-v2'...
     Reranking completado.
  6. Seleccionando los 3 mejores resultados finales.
     Doc 1 Final (ID 8): Reranker Score = -0.1245
     Doc 2 Final (ID 5): Reranker Score = -0.2842
     Doc 3 Final (ID 7): Reranker Score = -0.3471
--- Contexto Formateado Final (3 chunks):
Fuente: ID:8 | Contenido: Una segunda lente magnética, el
objetivo, da una imagen aumentada del objeto. Una tercera lente, el
ocular, aumenta de nuevo la imagen dada por la anterior. La imagen
final es proyectada sobre una pantalla o fotografiada. Los microscopios electrónicos permiten aumentos útiles que van

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://router.h

     Embedding obtenido.
  2. Realizando búsqueda FAISS (k=100)...
     Búsqueda FAISS -> 100 candidatos.
  3. Realizando búsqueda BM25 (k=100)...
     Búsqueda BM25 -> 100 candidatos.
  4. Fusionando resultados...
  INFO: Nombre Propio detectado -> Mayor peso BM25
     Total IDs candidatos únicos: 108
     100 candidatos seleccionados para reranking.
  5. Rerankeando con 'cross-encoder/ms-marco-MiniLM-L-12-v2'...
     Reranking completado.
  6. Seleccionando los 3 mejores resultados finales.
     Doc 1 Final (ID 14): Reranker Score = 2.5601
     Doc 2 Final (ID 36): Reranker Score = -0.2591
     Doc 3 Final (ID 5): Reranker Score = -0.4109
--- Contexto Formateado Final (3 chunks):
Fuente: ID:14 | Contenido: 6ª La evolución en los vegetales. 7ª La evolución en los animales. PRIMERAS ETAPAS DE LA EVOLUCIÓN BIOLÓGICA
1ª) LA EVOLUCIÓN QUÍMICA
La evolución química de los primeros organismos a partir de la materia inanimada se dio
siguiendo los siguientes pasos:
1º Síntesis y concentración 

Traceback (most recent call last):
  File "/tmp/ipykernel_4795/2384842375.py", line 120, in run_rag_based_task
    response = rag_chain.invoke({})
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/runnables/base.py", line 3034, in invoke
    input = context.run(step.invoke, input, config)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/language_models/chat_models.py", line 369, in invoke
    self.generate_prompt(
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/language_models/chat_models.py", line 946, in generate_prompt
    return self.generate(prompt_messages, stop=stop, callbacks=callbacks, **kwargs)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/language_models/chat_models.py", line 765, in generate
    self._generate_with_cache(
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/language_model

KeyboardInterrupt: 

In [ ]:
# --- Celda: Métricas Automáticas (igual que antes, pero aplicada a nuevos resultados) ---
import numpy as np
import language_tool_python

# ... (Inicialización de LanguageTool como antes) ...
# ... (Cálculo de 'grammar_errors' como antes) ...
# ... (Cálculo de 'timing_summary' para 'duration_seconds' como antes) ...

# Ahora el resumen de retrieval aplica a todas las tareas
retrieval_summary_all = results_df.groupby(['task_type', 'llm_provider'])['retrieval_duration_seconds'].agg(['mean', 'median', 'std', 'count']).reset_index()
print("\n--- Resumen de Tiempos de Retrieval (por Tarea y LLM) ---")
display(retrieval_summary_all)

# ... (Guardar CSV como antes) ...